# Импорт библиотек

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

from rdkit.Chem import Descriptors, MolFromSmiles, rdFingerprintGenerator as fp

from chython import smiles
from collections import defaultdict

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import pytorch_lightning as pyl
from rdkit import Chem

from torch.utils.data import Dataset, DataLoader, TensorDataset

import warnings
warnings.filterwarnings("ignore")


from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors
from rdkit.Chem.EState import EState_VSA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt


from sklearn.metrics import r2_score, mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors, AllChem, MACCSkeys
from typing import List, Dict, Optional
from sklearn.feature_selection import VarianceThreshold

import pandas as pd
import numpy as np
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# TODO добавить prepare
class Data:
    def __init__(self, path_train, path_test):
        self._train = Path(path_train)
        self._test = Path(path_test)

    def load(self):
        train_data = pd.read_csv(self._train)
        test_data = pd.read_csv(self._test)

        return train_data, test_data


data_loader = Data('final_train_data_04_14_201_new_alex.csv', 'final_test_data_04_14_201_new_alex.csv')
train_data, test_data = data_loader.load()
y = train_data['LogP']

train_data.head()

In [ ]:
train_data = pd.read_csv('final_train_data_04_14_201_new_alex.csv')
test_data = pd.read_csv('final_test_data_04_14_201_new_alex.csv')

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
list(train_data.columns)

In [ ]:
# all_descriptors = Descriptors.descList[:100]
# descriptor_names = [name for name, func in all_descriptors]

# def compute_descriptors(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         return [None] * len(all_descriptors)
#     return [func(mol) for _, func in all_descriptors]

# def add_descriptors(df):
#     desc_values = []
#     valid_indices = []
#     for i, smi in tqdm(enumerate(df['SMILES']), total=len(df)):
#         desc = compute_descriptors(smi)
#         if None not in desc:
#             desc_values.append(desc)
#             valid_indices.append(i)
#     desc_df = pd.DataFrame(desc_values, columns=descriptor_names)
#     df_clean = df.iloc[valid_indices].reset_index(drop=True)
#     df_clean = pd.concat([df_clean.reset_index(drop=True), desc_df], axis=1)
#     return df_clean

# train_data = add_descriptors(train_data)
# test_data = add_descriptors(test_data)

In [ ]:
train_data.head()

In [ ]:
test_data

In [ ]:
X_train = train_data.drop(columns=['LogP'])
y_train = train_data['LogP']
X_test = test_data

In [ ]:
# Проверми пропуски
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
# Заполним пропуски средним
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())

In [ ]:
# Проверми пропуски еще раз
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

catboost_regressor = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', verbose=0, random_seed=42)

param_catboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

grid_search_catboost = GridSearchCV(catboost_regressor, param_catboost, cv=5,  scoring='neg_root_mean_squared_error', n_jobs=-1)

grid_search_catboost.fit(X_train, y_train)

In [ ]:
grid_search_catboost.best_params_

In [ ]:
y_test_pred = best_model.predict(X_test)

In [ ]:
# Предсказание на тестовых данных
test_data = pd.read_csv('final_test_data_04_14_201_new_alex.csv')
predictions = best_model.predict(test_data)

# Сохранение в формате submission (как в примере с SVM)
submission_template = pd.read_csv('final_sample_submission80.csv')  # Используем тот же шаблон
submission = submission_template.copy()
submission['LogP'] = predictions
submission.to_csv('predict333.csv', index=False)

print("\nСохранено.")

**Результат модели на платформе: 0.691942**

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
import optuna
from optuna.exceptions import TrialPruned
import multiprocessing as mp
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time


def load_and_split_data(df: pd.DataFrame, target_col: str, test_size: float, random_state: int):
    """Загружает, разделяет признаки и цель, выполняет train/test split."""
    try:
        X = df.drop(columns=[target_col])
        y = df[target_col]
    except KeyError:
        print(f"Ошибка: Целевая колонка '{target_col}' не найдена в DataFrame.")
        raise

    stratify_option = None

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=stratify_option)

    print(f"Данные разделены (полные): X_train_full: {X_train.shape}, X_test: {X_test.shape}")

    return X_train, X_test, y_train, y_test, X.columns

def create_pipeline(trial: optuna.trial.Trial) -> Pipeline:
    """Создает пайплайн sklearn с гиперпараметрами, предложенными Optuna."""

    variance_threshold = trial.suggest_float("variance_threshold", 0.0, 0.1, log=False)
    #kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])
    kernel = trial.suggest_categorical("kernel", ["rbf", "poly"])

    C = trial.suggest_float("C", 1e-4, 1e2, log=True)

    if kernel == "rbf": # не учитываем kernel = 'linear' - с ним результат не высок
        gamma = trial.suggest_categorical("gamma_rbf", ["scale", "auto"])
        # gamma_val = trial.suggest_float("gamma_rbf_val", 1e-4, 1e1, log=True) # Альтернатива
        svm_params = {"kernel": kernel, "C": C, "gamma": gamma}
    elif kernel == "poly":
        degree = trial.suggest_int("degree", 2, 6)
        gamma = trial.suggest_categorical("gamma_poly", ["scale", "auto"])
        svm_params = {"kernel": kernel, "C": C, "degree": degree, "gamma": gamma}

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('variance_filter', VarianceThreshold(threshold=variance_threshold)),
        ('svm', SVR(**svm_params, cache_size=5000))
    ])
    return pipeline

def objective(trial: optuna.trial.Trial, X_train: pd.DataFrame, y_train: pd.Series, cv_folds: int, scoring: str) -> float:
    """Целевая функция для Optuna."""
    pipeline = create_pipeline(trial)
    try:
        scores = cross_val_score(
            pipeline, X_train, y_train,
            cv=KFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE),
            scoring=scoring, # здесь  'r_squared'
            n_jobs=-1
        )
        mean_score = np.mean(scores)

    except ValueError as e:
        print(f"Ошибка во время cross_val_score для триала {trial.number}: {e}")

        return -np.inf

    # Обработка NaN/inf в скорах
    if np.isnan(mean_score) or np.isinf(mean_score):
        print(f"Предупреждение: CV вернул NaN/inf для триала {trial.number}. Возвращаем плохое значение.")

        return -np.inf

    return mean_score

def train_final_model(best_params: dict, X_train_full: pd.DataFrame, y_train_full: pd.Series) -> Pipeline:
    """Обучает финальный пайплайн на ВСЕХ обучающих данных с лучшими параметрами."""
    print("\n Обучение финальной модели с лучшими параметрами на ПОЛНОМ обучающем датасете...")
    variance_threshold = best_params['variance_threshold']
    kernel = best_params['kernel']
    C = best_params['C']

    svm_params = {"kernel": kernel, "C": C}
    if kernel == "rbf":
        svm_params["gamma"] = best_params['gamma_rbf']
    elif kernel == "poly":
        svm_params["degree"] = best_params['degree']
        svm_params["gamma"] = best_params['gamma_poly']

    final_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('variance_filter', VarianceThreshold(threshold=variance_threshold)),
        ('svm', SVR(**svm_params, cache_size=5000))
    ])

    final_pipeline.fit(X_train_full, y_train_full)
    print("Финальная модель обучена.")

    try:
        variance_filter_step = final_pipeline.named_steps['variance_filter']
        feature_mask = variance_filter_step.get_support()
        n_features_selected = sum(feature_mask)
        print(f"Количество признаков после VarianceThreshold (threshold={variance_threshold:.4f}): {n_features_selected} / {X_train_full.shape[1]}")
    except Exception as e:
        print(f"Не удалось получить количество признаков после фильтрации: {e}")

    return final_pipeline

def evaluate_model(pipeline: Pipeline, X_test: pd.DataFrame, y_test: pd.Series):
    """Оценивает обученный пайплайн на тестовых данных и выводит метрики."""
    print("\nОценка качества лучшей модели на тестовой выборке...")
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred) # R2 считаем для отчета
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Coefficient of Determination (R²): {r2:.4f}")
    return y_pred, {'mse': mse, 'rmse': rmse, 'r2': r2}

def plot_results(y_test: pd.Series, y_pred: np.ndarray):
    """Строит график предсказанных vs реальных значений."""
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_test, y=y_pred, alpha=0.7)
    plt.xlabel("Actual LogP", fontsize=12)
    plt.ylabel("Predicted LogP", fontsize=12)
    plt.title("Optimized SVM Regression (Max R²): Predicted vs Actual LogP", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', linewidth=2)
    plt.show()

def plot_optuna_results(study: optuna.study.Study):
    """Визуализирует результаты оптимизации Optuna."""
    try:
        if optuna.visualization.is_available():
            fig1 = optuna.visualization.plot_optimization_history(study)
            fig1.show()

            fig2 = optuna.visualization.plot_param_importances(study)
            fig2.show()
        else:
            print("Optuna visualization is not available. Please install plotly.")
    except Exception as e:
        print(f"Ошибка при построении графиков Optuna: {e}")

TARGET_COLUMN = 'LogP'
TEST_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 5


SCORING = 'r2'

N_TRIALS = 100
OPTUNA_TIMEOUT = 10000

USE_SUBSET_FOR_OPTUNA = True
OPTUNA_SUBSET_FRACTION = 0.5
STRATIFY_SUBSET = False

desc_df = pd.read_csv('final_train_data_04_14_201_new_alex.csv')

X_train_full, X_test, y_train_full, y_test, feature_names = load_and_split_data(desc_df,
                                                                                TARGET_COLUMN,
                                                                                TEST_SIZE,
                                                                                RANDOM_STATE)

if USE_SUBSET_FOR_OPTUNA and OPTUNA_SUBSET_FRACTION < 1.0:
    print(f"\n Используется подвыборка ({OPTUNA_SUBSET_FRACTION * 100:.0f}%) для оптимизации Optuna.")
    stratify_option = None
    X_train_subset, _, y_train_subset, _ = train_test_split(X_train_full,
                                                            y_train_full,
                                                            train_size=OPTUNA_SUBSET_FRACTION,
                                                            random_state=RANDOM_STATE + 1,
                                                            stratify=stratify_option)

    print(f"Размер подвыборки для Optuna: X_train_subset: {X_train_subset.shape}")
    X_train_optuna = X_train_subset
    y_train_optuna = y_train_subset
else:
    print("\n Используется ПОЛНЫЙ обучающий набор данных для оптимизации Optuna.")
    X_train_optuna = X_train_full
    y_train_optuna = y_train_full

direction = "maximize"
study = optuna.create_study(direction=direction)

print(f"\nЗапуск оптимизации Optuna ({N_TRIALS} триалов, цель: максимизировать {SCORING})...")
start_time = time.time()

study.optimize(lambda trial: objective(trial, X_train_optuna, y_train_optuna, CV_FOLDS, SCORING),
               n_trials=N_TRIALS,
               timeout=OPTUNA_TIMEOUT,
               n_jobs=1)

end_time = time.time()
print(f"Оптимизация завершена за {end_time - start_time:.2f} секунд.")


print("\n Результаты оптимизации Optuna:")

print(f"Лучшее значение метрики ({SCORING}): {study.best_value:.4f}")


print("Лучшие гиперпараметры:")
best_params = study.best_params
for key, value in best_params.items():
    print(f"  {key}: {value}")

final_model_pipeline = train_final_model(best_params, X_train_full, y_train_full)


joblib.dump(final_model_pipeline, 'final_svm_optuna_pipeline_04_14_201_new_alex.joblib')
# joblib.dump(study, 'optuna_study.joblib')

y_pred_test, test_metrics = evaluate_model(final_model_pipeline, X_test, y_test)

plot_results(y_test, y_pred_test)
plot_optuna_results(study)

test_data = pd.read_csv('final_test_data_04_14_201_new_alex.csv')
predictions = final_model_pipeline.predict(test_data)
submission = pd.read_csv('final_sample_submission80.csv')
submission.LogP = predictions
submission.to_csv('prediction_04_14_201_new_alex.csv', index=False)


print("\nСкрипт успешно выполнен!")

## Тестирую модели на новых данных, обавление 200 фичей + новые

In [ ]:
train = pd.read_csv('train_data_clean_rdkit.csv')
test = pd.read_csv('test_rdkit_filtered.csv')

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

# Проверка на наличие SMILES
assert 'SMILES' in train.columns, "Нет столбца SMILES в train"
assert 'SMILES' in test.columns, "Нет столбца SMILES в test"

# Дескрипторы, которые мы добавим
extra_descriptors = {
    'MolMR': Descriptors.MolMR,
    'NumValenceElectrons': Descriptors.NumValenceElectrons,
    'MaxPartialCharge': Descriptors.MaxPartialCharge,
    'MinPartialCharge': Descriptors.MinPartialCharge,
    'NumRadicalElectrons': Descriptors.NumRadicalElectrons,
    'NumAromaticRings': rdMolDescriptors.CalcNumAromaticRings,
    'NumAliphaticRings': rdMolDescriptors.CalcNumAliphaticRings,
    'NumSaturatedRings': rdMolDescriptors.CalcNumSaturatedRings
}

# Функция вычисления признаков
def compute_features(smiles_list):
    descriptors = []
    morgan_fp = []
    maccs_fp = []

    for smi in tqdm(smiles_list, desc="Обработка молекул"):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            descriptors.append([np.nan]*len(extra_descriptors))
            morgan_fp.append([0]*256)
            maccs_fp.append([0]*167)
            continue

        # Основные дескрипторы
        desc_values = [func(mol) for func in extra_descriptors.values()]
        descriptors.append(desc_values)

        # Morgan Fingerprint
        fp = GetMorganFingerprintAsBitVect(mol, radius=2, nBits=256)
        morgan_fp.append(list(fp))

        # MACCS Keys
        maccs = MACCSkeys.GenMACCSKeys(mol)
        maccs_fp.append(list(maccs)[1:])  # первый элемент всегда 0

    desc_df = pd.DataFrame(descriptors, columns=extra_descriptors.keys())
    morgan_df = pd.DataFrame(morgan_fp, columns=[f'Morgan_{i}' for i in range(256)])
    maccs_df = pd.DataFrame(maccs_fp, columns=[f'MACCS_{i}' for i in range(1, 167)])


    return pd.concat([desc_df, morgan_df, maccs_df], axis=1)

# Применение к train и test
print("Вычисляем фичи для train...")
train_features = compute_features(train['SMILES'])
train_extended = pd.concat([train, train_features], axis=1)
train_extended.to_csv("train_extended.csv", index=False)

print("Вычисляем фичи для test...")
test_features = compute_features(test['SMILES'])
test_extended = pd.concat([test, test_features], axis=1)
test_extended.to_csv("test_extended.csv", index=False)

print("Файлы train_extended.csv и test_extended.csv сохранены!")


In [ ]:
train_data_new

In [ ]:
test_data_new

In [ ]:
X_train = train_data_new.drop(columns=['SMILES', 'LogP'])
y_train = train_data_new['LogP']
X_test = test_data_new.drop(columns=['ID', 'SMILES'])

In [ ]:
# Проверми пропуски
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
# Заполним пропуски средним
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())

In [ ]:
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

catboost_regressor = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', verbose=0, random_seed=42)

param_catboost = {
    'n_estimators': [200, 300, 350],
    'learning_rate': [0.01],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [3, 5]
}

grid_search_catboost2 = GridSearchCV(catboost_regressor, param_catboost, cv=5,  scoring='neg_root_mean_squared_error', n_jobs=-1)

grid_search_catboost2.fit(X_train, y_train)

In [ ]:
best_model22 = grid_search_catboost2.best_estimator_

In [ ]:
y_test_pred22 = best_model22.predict(X_test)

submission = pd.DataFrame({
    'ID': test_data_new['ID'],
    'LogP': y_test_pred22
})
submission.to_csv('sub333.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
best_model.fit(X_tr, y_tr)

In [ ]:
y_val_pred = best_model.predict(X_val)

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(y_val, y_val_pred)
print("RMSE на отложенной валидации:", rmse)

### Score модели в Leaderboard: **0.66085**

## **Тест №3. Добавление новых дескрипторов**

---

## **Тест RDKit**

**Добавим больше дескрипторов**

In [ ]:
train_data2 = pd.read_csv('/content/rdkit_standardized_data_without_duplicates.csv')
test_data2 = pd.read_csv('/content/test_rdkit_standardized_data_without_duplicates.csv')

In [ ]:
train_data2

In [ ]:
test_data2

In [ ]:
# Функция для безопасного преобразования SMILES
def clean_smiles(smiles):
    if pd.isna(smiles):
        return None
    try:
        return str(smiles).strip()
    except:
        return None

# Все возможные дескрипторы RDKit
all_descriptors = Descriptors.descList
descriptor_names = [name for name, func in all_descriptors]

def compute_descriptors(smiles):
    # Очистка и проверка SMILES
    clean_smi = clean_smiles(smiles)
    if clean_smi is None:
        return None

    mol = Chem.MolFromSmiles(clean_smi)
    if mol is None:
        return None

    desc_values = {}
    for name, func in all_descriptors:
        try:
            desc_values[name] = func(mol)
        except:
            desc_values[name] = None
    return desc_values

def add_descriptors(df):
    results = []
    valid_indices = []
    invalid_count = 0

    for i, smi in tqdm(enumerate(df['SMILES']), total=len(df)):
        desc = compute_descriptors(smi)
        if desc is not None:
            results.append(desc)
            valid_indices.append(i)
        else:
            invalid_count += 1

    print(f"\nПропущено {invalid_count} невалидных SMILES")

    # Создаем DataFrame и очищаем его
    desc_df = pd.DataFrame(results)

    # Удаляем полностью пустые столбцы
    desc_df = desc_df.dropna(axis=1, how='all')

    # Удаляем столбцы с постоянными значениями
    desc_df = desc_df.loc[:, desc_df.nunique() > 1]

    # Объединяем с исходными данными
    df_clean = df.iloc[valid_indices].reset_index(drop=True)
    df_clean = pd.concat([df_clean, desc_df], axis=1)

    actual_descriptors = desc_df.columns.tolist()
    print(f"Успешно добавлено {len(actual_descriptors)} дескрипторов")

    return df_clean, actual_descriptors

# Обработка данных
print("Обработка тренировочных данных...")
train_data2, train_descriptors = add_descriptors(train_data2)

print("\nОбработка тестовых данных...")
test_data2, _ = add_descriptors(test_data2)

# Сохраняем обработанные данные
train_data2.to_csv('train_data_processed.csv', index=False)
test_data2.to_csv('test_data_processed.csv', index=False)

In [ ]:
train_data2.head()

In [ ]:
test_data2.head()

In [ ]:
X_train = train_data2.drop(columns=['SMILES', 'LogP'])
y_train = train_data2['LogP']
X_test = test_data2.drop(columns=['ID', 'SMILES'])

In [ ]:
# Проверми пропуски
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
# Заполним пропуски средним
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())

In [ ]:
# Проверми пропуски еще раз
print(X_train.isna().sum().sum(), X_test.isna().sum().sum())

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

catboost_regressor = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', verbose=0, random_seed=42)

param_catboost = {
    'n_estimators': [200, 300, 350],
    'learning_rate': [0.01],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [3, 5]
}

grid_search_catboost2 = GridSearchCV(catboost_regressor, param_catboost, cv=5,  scoring='neg_root_mean_squared_error', n_jobs=-1)

grid_search_catboost2.fit(X_train, y_train)

In [ ]:
grid_search_catboost2.best_params_

In [ ]:
best_model2 = grid_search_catboost2.best_estimator_

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
best_model2.fit(X_tr, y_tr)

In [ ]:
y_val_pred = best_model2.predict(X_val)

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(y_val, y_val_pred)
print("RMSE на отложенной валидации:", rmse)

In [ ]:
y_test_pred2 = best_model2.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    'ID': test_data2['ID'],
    'LogP': y_test_pred2
})
submission.to_csv('1.csv', index=False)

### Score в Leaderboard: **0.65673**

---

## **Тест №4. Добавление новых фингерпринтов. Тест через стекинг**

---

In [ ]:
train_data3 = pd.read_csv('train_data.csv')
test_data3 = pd.read_csv('test_data.csv')

In [ ]:
# Определение ключевых дескрипторов
selected_descriptors = {
    # Физико-химические
    'MolWt': Descriptors.MolWt,
    'MolLogP': Descriptors.MolLogP,
    'MolMR': Descriptors.MolMR,
    'TPSA': Descriptors.TPSA,
    'LabuteASA': Descriptors.LabuteASA,

    # Водородные связи
    'NumHDonors': Lipinski.NumHDonors,
    'NumHAcceptors': Lipinski.NumHAcceptors,

    # Конституционные
    'HeavyAtomCount': Descriptors.HeavyAtomCount,
    'NumRotatableBonds': Descriptors.NumRotatableBonds,
    'FractionCSP3': Descriptors.FractionCSP3,
    'RingCount': rdMolDescriptors.CalcNumRings,

    # EState индексы
    'EState_VSA1': EState_VSA.EState_VSA1,
    'EState_VSA2': EState_VSA.EState_VSA2,

    # Дополнительные
    'BalabanJ': Chem.GraphDescriptors.BalabanJ,
    'BertzCT': Chem.GraphDescriptors.BertzCT
}

In [ ]:
# 2. Функция для вычисления дескрипторов и отпечатков
def compute_features(smiles_list, n_bits=256):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    features = []

    for mol in mols:
        if mol is None:
            features.append([None]*len(selected_descriptors) + [0]*n_bits)
            continue

        # Вычисляем дескрипторы
        desc_values = []
        for name, func in selected_descriptors.items():
            try:
                desc_values.append(func(mol))
            except:
                desc_values.append(None)

        # Вычисляем отпечатки Моргана
        fp = list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits))

        features.append(desc_values + fp)

    return pd.DataFrame(features,
                       columns=list(selected_descriptors.keys()) + [f'FP_{i}' for i in range(n_bits)])

In [ ]:
# 3. Вычисление фичей
print("Вычисление фичей для тренировочных данных...")
train_features = compute_features(train_data3['SMILES'])
train_features['LogP'] = train_data3['LogP'].values

print("\nВычисление фичей для тестовых данных...")
test_features = compute_features(test_data3['SMILES'])

In [ ]:
# 4. Подготовка данных
X_train = train_features.drop('LogP', axis=1)
y_train = train_features['LogP']
X_test = test_features

# Разделение на группы фичей
desc_cols = list(selected_descriptors.keys())
fp_cols = [col for col in X_train.columns if col.startswith('FP_')]

In [ ]:
# 5. Создание пайплайна
preprocessor = ColumnTransformer([
    ('desc', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), desc_cols),
    ('fp', 'passthrough', fp_cols)
])

# Базовые модели
base_models = [
    ('catboost', CatBoostRegressor(
        iterations=1000,
        depth=7,
        l2_leaf_reg=5,
        learning_rate=0.03,
        random_seed=42,
        verbose=0
    )),
    ('xgb', XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
]

# Мета-модель
meta_model = ElasticNet(alpha=0.001, l1_ratio=0.7)

# Полный пайплайн
final_model = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', VarianceThreshold(threshold=0.15)),
    ('stacking', StackingRegressor(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5,
        n_jobs=-1
    ))
])

In [ ]:
X_train

In [ ]:
# 6. Обучение модели
final_model.fit(X_train, y_train)

In [ ]:
# 7. Предсказание и сохранение результатов
test_pred = final_model.predict(X_test)

submission = pd.DataFrame({
    'ID': test_data3['ID'],
    'LogP': test_pred
})
submission.to_csv('submission333.csv', index=False)

print("\nСохранено")

In [ ]:
X_train

In [ ]:
X_test

Добавление новых фичей

In [ ]:
from tqdm.notebook import tqdm as tqdm_notebook # Используем для ноутбуков
from tqdm import tqdm as tqdm_base # Для обычного скрипта
import sys

# --- 0. Настройка ---
# Игнорирование предупреждений
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
RDLogger.DisableLog('rdApp.*')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Выбор TQDM
try: from tqdm.notebook import tqdm as tqdm_func
except ImportError: from tqdm import tqdm as tqdm_func

# Конфигурация путей (!!! ПРОВЕРЬ ИХ !!!)
SOURCE_TRAIN_PATH = 'train_data_clean_rdkit (2).csv' # Вход: SMILES, LogP
SOURCE_TEST_PATH = 'test_data.csv'                  # Вход: ID, SMILES
SAMPLE_SUBMISSION_PATH = 'final_sample_submission80.csv' # Для получения ID теста, если их нет в test_data.csv

OUTPUT_TRAIN_EXTENDED_PATH = 'train_features_extended.csv' # Выход: Расширенные признаки + LogP
OUTPUT_TEST_EXTENDED_PATH = 'test_features_extended.csv'   # Выход: Расширенные признаки + ID
OUTPUT_IMPUTER_PATH = 'feature_imputer.pkl'              # Сохраненный импутер

# Параметры фингерпринтов
ECFP_RADIUS = 2
ECFP_NBITS = 2048
FP_GENERATOR = GetMorganGenerator(radius=ECFP_RADIUS, fpSize=ECFP_NBITS) # Создаем генератор один раз

# Имена колонок
SMILES_COL = 'SMILES'
TARGET_COL = 'LogP'
ID_COL = 'ID'

# --- 1. Функции ---

def standardize_mol(mol):
    """Стандартизирует молекулу RDKit."""
    if mol is None: return None
    try:
        # Remove fragments
        clean_mol = rdMolStandardize.Cleanup(mol)
        # Get parent fragment
        parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
        # Neutralize charges
        uncharger = rdMolStandardize.Uncharger() # Handles multiple fragments if necessary
        uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)
        # Get canonical tautomer
        te = rdMolStandardize.TautomerEnumerator() # Default parameters are usually good
        canon_tautomer_mol = te.Canonicalize(uncharged_parent_clean_mol)
        return canon_tautomer_mol
    except Exception as e:
        # logging.warning(f"Standardization failed for a molecule: {e}")
        return None # Возвращаем None при любой ошибке стандартизации

def calculate_all_features(smiles: str, descriptor_list: list) -> tuple[Optional[np.ndarray], Optional[list]]:
    """
    Рассчитывает RDKit дескрипторы, ECFP и MACCS для одного SMILES.
    Возвращает numpy массив признаков и список их имен.
    """
    mol = Chem.MolFromSmiles(smiles)
    std_mol = standardize_mol(mol) # Стандартизируем

    if std_mol is None:
        # logging.warning(f"Could not process or standardize SMILES: {smiles}")
        return None, None # Возвращаем None, если молекула невалидна или не стандартизируется

    feature_values = []
    feature_names = []

    # 1. RDKit Descriptors
    for desc_name, desc_func in descriptor_list:
        try:
            val = desc_func(std_mol)
            # Заменяем бесконечности и не-числа на NaN
            if isinstance(val, (int, float)) and np.isfinite(val):
                feature_values.append(float(val))
            else:
                feature_values.append(np.nan)
        except Exception:
            feature_values.append(np.nan)
        feature_names.append(desc_name)

    # 2. ECFP (Morgan) Fingerprint
    ecfp_features = np.zeros((ECFP_NBITS,), dtype=np.int8) # Используем int8 для экономии памяти
    try:
        # fp = AllChem.GetMorganFingerprintAsBitVect(std_mol, ECFP_RADIUS, nBits=ECFP_NBITS)
        fp = FP_GENERATOR.GetFingerprintAsNumPy(std_mol) # Используем генератор
        # DataStructs.ConvertToNumpyArray(fp, ecfp_features) # Старый способ
        ecfp_features = fp.astype(np.int8)
    except Exception as e:
        # logging.warning(f"ECFP calculation failed for SMILES: {smiles} - {e}")
        pass # Оставляем нулевой вектор
    feature_values.extend(ecfp_features.tolist())
    feature_names.extend([f'ECFP_{i}' for i in range(ECFP_NBITS)])

    # 3. MACCS Keys Fingerprint
    maccs_features = np.zeros((167,), dtype=np.int8) # MACCS всегда 167 бит
    try:
        maccs_fp = AllChem.GetMACCSKeysFingerprint(std_mol)
        # DataStructs.ConvertToNumpyArray(maccs_fp, maccs_features) # Старый способ
        # Новый способ: прямое преобразование
        for i in range(167):
             if maccs_fp.GetBit(i):
                 maccs_features[i] = 1
    except Exception as e:
        # logging.warning(f"MACCS calculation failed for SMILES: {smiles} - {e}")
        pass # Оставляем нулевой вектор
    feature_values.extend(maccs_features.tolist())
    feature_names.extend([f'MACCS_{i}' for i in range(1, 168)]) # Нумерация с 1 до 167

    return np.array(feature_values, dtype=np.float32), feature_names


# --- 2. Основная Обработка ---

# Получаем список RDKit дескрипторов один раз
rdkit_descriptor_list = Descriptors.descList
logging.info(f"Using {len(rdkit_descriptor_list)} RDKit descriptors.")

# Определяем общую длину вектора признаков (приблизительно)
# Это нужно для создания массива NaN в случае ошибки
num_rdkit_desc = len(rdkit_descriptor_list)
num_ecfp_bits = ECFP_NBITS
num_maccs_bits = 167
total_feature_length = num_rdkit_desc + num_ecfp_bits + num_maccs_bits
logging.info(f"Total expected feature length: {total_feature_length}")

all_feature_names = None # Определим по первому успешному расчету

# --- Обработка Тренировочного Набора ---
logging.info(f"Processing Training Data from: {SOURCE_TRAIN_PATH}...")
try:
    df_train_source = pd.read_csv(SOURCE_TRAIN_PATH)
    if SMILES_COL not in df_train_source.columns or TARGET_COL not in df_train_source.columns:
        raise ValueError(f"Missing '{SMILES_COL}' or '{TARGET_COL}' in {SOURCE_TRAIN_PATH}")
    df_train_source.dropna(subset=[SMILES_COL, TARGET_COL], inplace=True)
    logging.info(f"Loaded {len(df_train_source)} training SMILES.")
except Exception as e:
    logging.error(f"Failed to load training data: {e}"); sys.exit(1)

train_features_list = []
train_logp_list = []
failed_train_smiles = 0

pbar_train = tqdm_func(df_train_source.iterrows(), total=len(df_train_source), desc="Calculating Train Features")
for index, row in pbar_train:
    smiles = row[SMILES_COL]
    logp = row[TARGET_COL]
    features, current_names = calculate_all_features(smiles, rdkit_descriptor_list)

    if features is not None:
        if all_feature_names is None: # Получаем имена фичей при первой удаче
            all_feature_names = current_names
            # Перепроверяем длину
            if len(all_feature_names) != total_feature_length:
                 logging.warning(f"Feature name length ({len(all_feature_names)}) mismatch with expected ({total_feature_length}). Using actual.")
                 total_feature_length = len(all_feature_names)

        # Проверяем консистентность длины вектора
        if len(features) == total_feature_length:
            train_features_list.append(features)
            train_logp_list.append(logp)
        else:
            logging.warning(f"Feature length mismatch for train SMILES {smiles} (got {len(features)}, expected {total_feature_length}). Skipping.")
            failed_train_smiles += 1
    else:
        failed_train_smiles += 1

logging.info(f"Finished processing train data. Success: {len(train_features_list)}, Failed: {failed_train_smiles}")
if not train_features_list:
    logging.error("No features were successfully calculated for the training set. Exiting."); sys.exit(1)

# Создаем DataFrame для трейна
X_train_ext = pd.DataFrame(train_features_list, columns=all_feature_names)
y_train_ext = pd.Series(train_logp_list, name=TARGET_COL)


# --- Обработка Тестового Набора ---
logging.info(f"Processing Test Data from: {SOURCE_TEST_PATH}...")
try:
    df_test_source = pd.read_csv(SOURCE_TEST_PATH)
    if SMILES_COL not in df_test_source.columns:
        raise ValueError(f"Missing '{SMILES_COL}' in {SOURCE_TEST_PATH}")
    # Проверяем наличие ID, если нет - используем sample submission
    if ID_COL not in df_test_source.columns:
         logging.warning(f"'{ID_COL}' not found in test data. Loading IDs from {SAMPLE_SUBMISSION_PATH}...")
         df_sample_sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
         if ID_COL not in df_sample_sub.columns: raise ValueError(f"'{ID_COL}' not found in sample submission either!")
         if len(df_sample_sub) != len(df_test_source): raise ValueError("Length mismatch between test data and sample submission.")
         test_ids = df_sample_sub[ID_COL].values
         df_test_source[ID_COL] = test_ids # Добавляем ID
    else:
         test_ids = df_test_source[ID_COL].values # Используем ID из test_data

    df_test_source.dropna(subset=[SMILES_COL], inplace=True) # Удаляем только если SMILES пустой
    logging.info(f"Loaded {len(df_test_source)} test SMILES.")

except Exception as e:
    logging.error(f"Failed to load test data: {e}"); sys.exit(1)

test_features_list = []
test_ids_list = [] # Сохраняем ID для строк, где фичи посчитались
failed_test_smiles = 0
fallback_feature_vector = np.full(total_feature_length, np.nan, dtype=np.float32) # Вектор для ошибок

pbar_test = tqdm_func(df_test_source.iterrows(), total=len(df_test_source), desc="Calculating Test Features")
for index, row in pbar_test:
    smiles = row[SMILES_COL]
    current_id = row[ID_COL]
    features, _ = calculate_all_features(smiles, rdkit_descriptor_list)

    if features is not None and len(features) == total_feature_length:
        test_features_list.append(features)
    else:
        # logging.warning(f"Feature calculation failed or length mismatch for test SMILES {smiles}. Using NaN vector.")
        test_features_list.append(fallback_feature_vector) # Добавляем NaN вектор
        failed_test_smiles += 1
    test_ids_list.append(current_id) # Добавляем ID в любом случае, чтобы сохранить порядок

logging.info(f"Finished processing test data. Success/Fallback: {len(test_features_list)}, Failed to generate vector (should be 0): {failed_test_smiles}")
if not test_features_list:
    logging.error("No features were calculated for the test set. Exiting."); sys.exit(1)

# Создаем DataFrame для теста
X_test_ext = pd.DataFrame(test_features_list, columns=all_feature_names)


# --- 3. Импутация NaN ---
logging.info("Handling NaN values with Median Imputation...")
nan_counts_train = X_train_ext.isnull().sum()
cols_with_nan_train = nan_counts_train[nan_counts_train > 0]
if not cols_with_nan_train.empty:
    logging.info(f"NaNs found in {len(cols_with_nan_train)} training features columns.")
    # print(cols_with_nan_train) # Можно раскомментировать для детального просмотра

    imputer = SimpleImputer(strategy='median') # Медиана более устойчива к выбросам

    logging.info("Fitting imputer on training data...")
    X_train_imputed = imputer.fit_transform(X_train_ext)
    X_train_ext = pd.DataFrame(X_train_imputed, columns=all_feature_names, index=X_train_ext.index)
    logging.info("Training data imputed.")

    logging.info("Transforming test data with fitted imputer...")
    nan_counts_test_before = X_test_ext.isnull().sum().sum()
    if nan_counts_test_before > 0:
         X_test_imputed = imputer.transform(X_test_ext)
         X_test_ext = pd.DataFrame(X_test_imputed, columns=all_feature_names, index=X_test_ext.index)
         logging.info(f"Test data imputed. {nan_counts_test_before} NaNs filled.")
    else:
         logging.info("No NaNs found in test data before imputation.")

    # Сохраняем импутер
    joblib.dump(imputer, OUTPUT_IMPUTER_PATH)
    logging.info(f"Imputer saved to {OUTPUT_IMPUTER_PATH}")

else:
    logging.info("No NaN values found in training features. Skipping imputation.")
    # Проверяем NaN в тесте на всякий случай
    nan_counts_test = X_test_ext.isnull().sum().sum()
    if nan_counts_test > 0:
         logging.warning(f"NaNs found in TEST data ({nan_counts_test}) but not in TRAIN data. Filling with 0.")
         X_test_ext.fillna(0, inplace=True)


# --- 4. Сохранение Результатов ---

# Собираем финальные датафреймы
final_train_df = pd.concat([X_train_ext, y_train_ext], axis=1)
final_test_df = X_test_ext.copy()
final_test_df[ID_COL] = test_ids_list # Добавляем ID

# Перемещаем колонку ID в начало тестового DataFrame
cols = [ID_COL] + [col for col in final_test_df.columns if col != ID_COL]
final_test_df = final_test_df[cols]

logging.info(f"Final Train DataFrame shape: {final_train_df.shape}")
logging.info(f"Final Test DataFrame shape: {final_test_df.shape}")

logging.info(f"Saving extended training features to: {OUTPUT_TRAIN_EXTENDED_PATH}")
final_train_df.to_csv(OUTPUT_TRAIN_EXTENDED_PATH, index=False)

logging.info(f"Saving extended test features to: {OUTPUT_TEST_EXTENDED_PATH}")
final_test_df.to_csv(OUTPUT_TEST_EXTENDED_PATH, index=False)

logging.info("--- Feature Engineering Script Finished Successfully ---")

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

TRAIN_DATA_PATH = 'train_features_extended.csv'         # <<< Расширенные признаки трейна
TEST_DATA_PATH = 'test_features_extended.csv'           # <<< Расширенные признаки теста
SAMPLE_SUBMISSION_PATH = 'final_sample_submission80.csv' # Пример сабмишена

TARGET_COLUMN = 'LogP'
ID_COLUMN = 'ID' # ID должен быть в test_features_extended.csv
RANDOM_STATE = 42
CV_FOLDS = 5 # 5 или 10
N_TRIALS = 200 # Количество итераций Optuna (увеличь при наличии времени, >150)
OPTUNA_TIMEOUT = None # В секундах, None - без таймаута
SCORING = 'neg_root_mean_squared_error' # Цель Optuna - минимизация RMSE
OPTUNA_DIRECTION = "maximize" # Максимизируем neg_rmse
VT_THRESHOLD = 0.001 # Начальный порог для VarianceThreshold
SFM_THRESHOLD = 'median' # Порог для SelectFromModel ('mean', 'median', или число, например, 1e-5)


OUTPUT_DIR = Path('./lgbm_extended_fs_output')
OUTPUT_DIR.mkdir(exist_ok=True)
OPTUNA_DB_PATH = f"sqlite:///{OUTPUT_DIR}/optuna_lgbm_ext_fs.db"
STUDY_NAME = "lgbm_ext_fs_optimization_v1"
OUTPUT_MODEL_PATH = OUTPUT_DIR / 'final_lgbm_ext_fs_pipeline.pkl'
OUTPUT_SUBMISSION_PATH = OUTPUT_DIR / 'submission_lgbm_ext_fs.csv'
SELECTED_FEATURES_PATH = OUTPUT_DIR / 'selected_features.txt' # Сохраним список отобранных фичей



def load_data(file_path: str, target_col: Optional[str] = None):
    """Загружает данные."""
    logging.info(f"Загрузка данных из {file_path}...")
    try:
        df = pd.read_csv(file_path)
        logging.info(f"Загружен файл: {file_path}, форма: {df.shape}")
        if target_col and target_col not in df.columns:
            raise ValueError(f"Целевая колонка '{target_col}' не найдена.")
        if ID_COLUMN not in df.columns and target_col is None: # Проверка ID в тестовом файле
             raise ValueError(f"Колонка ID '{ID_COLUMN}' не найдена в тестовом файле {file_path}.")
        return df
    except FileNotFoundError: logging.error(f"Ошибка: Файл '{file_path}' не найден."); sys.exit(1)
    except ValueError as e: logging.error(f"Ошибка данных: {e}"); sys.exit(1)
    except Exception as e: logging.error(f"Ошибка загрузки файла {file_path}: {e}"); sys.exit(1)

def create_pipeline_optuna(trial: optuna.trial.Trial) -> Pipeline:
    """Создает пайплайн для Optuna (БЕЗ отбора признаков внутри)."""
    # Параметры препроцессинга (если нужен)
    scaler_type = trial.suggest_categorical("scaler", ["StandardScaler", "None"])

    preprocessing_steps = []
    if scaler_type == "StandardScaler":
        preprocessing_steps.append(('scaler', StandardScaler()))

    # Параметры LightGBM
    lgbm_params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True), # Уменьшил верхнюю границу LR
        'n_estimators': trial.suggest_int('n_estimators', 500, 3000), # Можно увеличить n_estimators
        'num_leaves': trial.suggest_int('num_leaves', 20, 150), # Увеличил диапазон
        'max_depth': trial.suggest_int('max_depth', 5, 20), # Увеличил диапазон
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-7, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-7, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100), # Увеличил диапазон
    }
    model = lgb.LGBMRegressor(**lgbm_params)
    pipeline = Pipeline(preprocessing_steps + [('lgbm', model)])
    return pipeline

def objective(trial: optuna.trial.Trial, X_tr: pd.DataFrame, y_tr: pd.Series, cv_folds: int, scoring: str) -> float:
    """Целевая функция для Optuna."""
    pipeline = create_pipeline_optuna(trial)
    try:
        cv = KFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE + trial.number) # Меняем random_state для CV
        scores = cross_val_score(pipeline, X_tr, y_tr, cv=cv, scoring=scoring, n_jobs=-1)
        mean_score = np.mean(scores)
        if np.isnan(mean_score) or np.isinf(mean_score): raise ValueError("CV score is NaN/inf")
    except Exception as e:
        logging.warning(f"Ошибка CV для триала {trial.number}: {e}")
        return -np.inf # Возвращаем худшее значение для максимизации
    return mean_score

def train_final_model(best_params: dict, X_train_full_sel: pd.DataFrame, y_train_full: pd.Series) -> Pipeline:
    """Обучает финальный пайплайн на ВСЕХ отобранных обучающих данных."""
    logging.info("--- Обучение финальной модели с лучшими параметрами ---")
    # Воссоздаем пайплайн препроцессинга
    preprocessing_steps = []
    scaler_type = best_params.get("scaler", "None") # Дефолт None, если не было в HPO
    if scaler_type == "StandardScaler":
        preprocessing_steps.append(('scaler', StandardScaler()))
        logging.info("Using StandardScaler in final model.")
    else:
        logging.info("Scaler not used in final model.")

    # Воссоздаем параметры LightGBM
    lgbm_params = {
        'objective': 'regression_l1', 'metric': 'rmse', 'random_state': RANDOM_STATE, 'n_jobs': -1,
        'learning_rate': best_params['learning_rate'], 'n_estimators': best_params['n_estimators'],
        'num_leaves': best_params['num_leaves'], 'max_depth': best_params['max_depth'],
        'reg_alpha': best_params['reg_alpha'], 'reg_lambda': best_params['reg_lambda'],
        'colsample_bytree': best_params['colsample_bytree'], 'subsample': best_params['subsample'],
        'min_child_samples': best_params['min_child_samples'],
    }
    final_model = lgb.LGBMRegressor(**lgbm_params)
    final_pipeline = Pipeline(preprocessing_steps + [('lgbm', final_model)])

    logging.info("Обучение на полном ОТОБРАННОМ обучающем датасете...")
    start_time = time.time()
    final_pipeline.fit(X_train_full_sel, y_train_full)
    end_time = time.time()
    logging.info(f"Финальная модель обучена за {end_time - start_time:.2f} секунд.")
    logging.info(f"Количество признаков, использованных моделью: {X_train_full_sel.shape[1]}")

    # Вывод важности признаков
    try:
        lgbm_step = final_pipeline.named_steps['lgbm']
        if hasattr(lgbm_step, 'feature_importances_'):
            importances = lgbm_step.feature_importances_
            feature_names_ = X_train_full_sel.columns.tolist() # Используем имена отобранных признаков
            if len(importances) == len(feature_names_):
                feature_importance_df = pd.DataFrame({'feature': feature_names_, 'importance': importances})
                feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
                logging.info("\nТоп-10 самых важных признаков (финальная модель):")
                logging.info("\n" + feature_importance_df.head(10).to_string())
            else:
                logging.warning("Несовпадение кол-ва важностей и признаков.")
    except Exception as e:
        logging.warning(f"Не удалось получить/вывести важность признаков: {e}")

    return final_pipeline

def evaluate_model(pipeline: Pipeline, X_test_sel: pd.DataFrame, y_test: pd.Series):
    """Оценивает пайплайн на ОТОБРАННЫХ тестовых данных."""
    logging.info("--- Оценка качества финальной модели на ЛОКАЛЬНОЙ тестовой выборке ---")
    try:
        y_pred = pipeline.predict(X_test_sel)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        logging.info(f"Mean Squared Error (MSE): {mse:.5f}")
        logging.info(f"Root Mean Squared Error (RMSE): {rmse:.5f}") # <<< ВАЖНАЯ МЕТРИКА
        logging.info(f"Coefficient of Determination (R²): {r2:.5f}")
        return y_pred, {'mse': mse, 'rmse': rmse, 'r2': r2}
    except Exception as e:
        logging.error(f"Ошибка при оценке модели: {e}")
        return None, {}

def create_submission(pipeline: Pipeline, test_data_path: str, selected_feature_names: list,
                      sample_submission_path: str, output_path: Path, id_col: str, target_col: str):
    """Делает предсказания на тестовом наборе (ИСПОЛЬЗУЯ ОТОБРАННЫЕ ПРИЗНАКИ) и сохраняет файл."""
    logging.info("--- Создание файла для отправки ---")
    try:
        test_df_full = pd.read_csv(test_data_path) # Загружаем ПОЛНЫЙ тестовый файл с РАСШИРЕННЫМИ признаками
        logging.info(f"Загружен полный тестовый набор: {test_df_full.shape}")

        # --- ВАЖНО: Отбираем только НУЖНЫЕ признаки ---
        if not all(f in test_df_full.columns for f in selected_feature_names):
             missing_cols = [f for f in selected_feature_names if f not in test_df_full.columns]
             raise ValueError(f"Не найдены все отобранные признаки в тестовом файле! Отсутствуют: {missing_cols}")

        X_test_submission_sel = test_df_full[selected_feature_names]
        logging.info(f"Отобраны признаки для предсказания: {X_test_submission_sel.shape}")

        # Проверка и обработка NaN (на всякий случай, хотя импутер должен был сработать)
        if X_test_submission_sel.isnull().sum().sum() > 0:
             logging.warning("Найдены NaN в отобранных тестовых признаках! Попытка заполнить нулями...")
             X_test_submission_sel = X_test_submission_sel.fillna(0) # Простая замена

        logging.info("Генерация предсказаний...")
        predictions = pipeline.predict(X_test_submission_sel)

        # Создание DataFrame для submission
        submission_df = pd.DataFrame({ id_col: test_df_full[id_col], target_col: predictions })

        # Сопоставление с Sample Submission для правильного порядка и набора ID
        df_sample = pd.read_csv(sample_submission_path)[[id_col]]
        final_submission_df = df_sample.merge(submission_df, on=id_col, how='left')

        if final_submission_df[target_col].isnull().any():
            logging.warning("NaN в итоговом сабмишене после merge! Проверьте ID.")
        if len(final_submission_df) != len(df_sample):
             logging.warning("Размер итогового сабмишена не совпадает с sample submission!")

        final_submission_df[[id_col, target_col]].to_csv(output_path, index=False)
        logging.info(f"Файл с предсказаниями сохранен в: {output_path}")
        logging.info("Пример предсказаний:\n" + final_submission_df.head().to_string())

    except FileNotFoundError: logging.error(f"Ошибка: Файл не найден при создании сабмишена."); sys.exit(1)
    except ValueError as e: logging.error(f"Ошибка данных при создании сабмишена: {e}"); sys.exit(1)
    except Exception as e: logging.error(f"Ошибка при создании файла submission: {e}"); sys.exit(1)

if __name__ == "__main__":

    # --- 2.1 Загрузка Полных Данных ---
    df_train_ext = load_data(TRAIN_DATA_PATH, target_col=TARGET_COLUMN)
    df_test_ext = load_data(TEST_DATA_PATH) # ID должен быть там

    X_full_train = df_train_ext.drop(columns=[TARGET_COLUMN])
    y_full_train = df_train_ext[TARGET_COLUMN]
    X_full_test  = df_test_ext.drop(columns=[ID_COLUMN]) # Убираем ID для обработки
    test_ids     = df_test_ext[ID_COLUMN].values         # Сохраняем ID

    # Проверка совпадения колонок (кроме таргета/ID)
    train_cols = set(X_full_train.columns)
    test_cols = set(X_full_test.columns)
    if train_cols != test_cols:
        logging.error("Колонки в трейне и тесте (расширенные признаки) не совпадают!")
        logging.error(f"Только в трейне: {train_cols - test_cols}")
        logging.error(f"Только в тесте: {test_cols - train_cols}")
        sys.exit(1)
    feature_names_initial = list(X_full_train.columns) # Имена всех исходных расширенных признаков

    # --- 2.2 Отбор Признаков ---
    logging.info("--- Выполнение отбора признаков ---")
    start_fs_time = time.time()

    # Шаг 1: Variance Threshold
    logging.info(f"Применение VarianceThreshold (threshold={VT_THRESHOLD})...")
    vt = VarianceThreshold(threshold=VT_THRESHOLD)
    try:
        X_train_vt = vt.fit_transform(X_full_train)
        X_test_vt = vt.transform(X_full_test)
        selected_features_vt_mask = vt.get_support()
        selected_feature_names_vt = [name for i, name in enumerate(feature_names_initial) if selected_features_vt_mask[i]]
        num_removed_vt = len(feature_names_initial) - len(selected_feature_names_vt)
        logging.info(f"VarianceThreshold удалил {num_removed_vt} признаков.")
        if not selected_feature_names_vt: raise ValueError("VarianceThreshold удалил все признаки!")
        # Обновляем DataFrame'ы
        X_train_processed = pd.DataFrame(X_train_vt, columns=selected_feature_names_vt, index=X_full_train.index)
        X_test_processed = pd.DataFrame(X_test_vt, columns=selected_feature_names_vt, index=X_full_test.index)
        logging.info(f"Признаков после VarianceThreshold: {X_train_processed.shape[1]}")
    except Exception as e:
        logging.error(f"Ошибка на этапе VarianceThreshold: {e}"); sys.exit(1)

    # Шаг 2: SelectFromModel (LGBM)
    logging.info(f"Применение SelectFromModel(LGBM) (threshold={SFM_THRESHOLD})...")
    try:
        # Используем простые параметры для модели-селектора
        selector_model = lgb.LGBMRegressor(
            objective='regression_l1', random_state=RANDOM_STATE, n_jobs=-1,
            n_estimators=200, learning_rate=0.05, num_leaves=31 # Простые параметры
        )
        selector = SelectFromModel(
            selector_model,
            threshold=SFM_THRESHOLD,
            # max_features=SFM_MAX_FEATURES, # Используй ИЛИ threshold ИЛИ max_features
            prefit=False
        )
        selector.fit(X_train_processed, y_full_train) # Обучаем селектор
        selected_features_final_mask = selector.get_support()
        selected_feature_names_final = X_train_processed.columns[selected_features_final_mask].tolist()
        num_selected_sfm = len(selected_feature_names_final)
        logging.info(f"SelectFromModel отобрал {num_selected_sfm} признаков.")
        if not selected_feature_names_final: raise ValueError("SelectFromModel не отобрал ни одного признака!")

        # --- Финальные отобранные признаки ---
        X_train_selected = X_train_processed[selected_feature_names_final]
        X_test_selected = X_test_processed[selected_feature_names_final] # Отбираем те же признаки из теста

        # Сохраняем список отобранных признаков
        with open(SELECTED_FEATURES_PATH, 'w') as f:
            for feature_name in selected_feature_names_final:
                f.write(f"{feature_name}\n")
        logging.info(f"Список отобранных признаков сохранен в: {SELECTED_FEATURES_PATH}")

    except Exception as e:
        logging.error(f"Ошибка на этапе SelectFromModel: {e}"); sys.exit(1)

    logging.info(f"Отбор признаков завершен за {time.time() - start_fs_time:.2f} секунд.")
    logging.info(f"Итоговое количество признаков для моделирования: {X_train_selected.shape[1]}")

    # --- 2.3 Локальный Train/Test Split (на отобранных признаках) ---
    logging.info("Создание локального train/test split для оценки...")
    X_train_local, X_test_local, y_train_local, y_test_local = train_test_split(
        X_train_selected, # Используем отобранные!
        y_full_train,     # Соответствующий таргет
        test_size=0.2,    # Стандартный размер для локальной оценки
        random_state=RANDOM_STATE
    )
    logging.info(f"Размеры локальных выборок: X_train={X_train_local.shape}, X_test={X_test_local.shape}")

    # --- 2.4 Запуск Optuna HPO ---
    logging.info("--- Запуск оптимизации Optuna ---")
    study = optuna.create_study(
        study_name=STUDY_NAME,
        storage=OPTUNA_DB_PATH,
        direction=OPTUNA_DIRECTION,
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        load_if_exists=True # Позволяет продолжить оптимизацию
    )

    # Определяем, сколько триалов осталось запустить
    completed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    trials_to_run = N_TRIALS - completed_trials
    logging.info(f"Всего задано триалов: {N_TRIALS}, уже завершено: {completed_trials}")

    if trials_to_run > 0:
        logging.info(f"Запускаем {trials_to_run} новых триалов...")
        start_optuna_time = time.time()
        try:
            study.optimize(
                lambda trial: objective(trial, X_train_local, y_train_local, CV_FOLDS, SCORING), # Оптимизация на локальном трейне
                n_trials=trials_to_run,
                timeout=OPTUNA_TIMEOUT,
                n_jobs=1, # Optuna в 1 поток, CV распараллеливается внутри
                show_progress_bar=True
            )
            logging.info(f"Сессия оптимизации Optuna завершена за {time.time() - start_optuna_time:.2f} секунд.")
        except KeyboardInterrupt: logging.info("Оптимизация прервана пользователем.")
        except Exception as e: logging.error(f"Ошибка во время оптимизации Optuna: {e}")
    else:
        logging.info("Заданное количество триалов Optuna уже выполнено.")

    # --- 2.5 Обучение и Оценка Финальной Модели ---
    if study.best_trial:
        logging.info("--- Результаты оптимизации Optuna ---")
        best_params = study.best_params
        best_cv_score = study.best_value
        best_cv_rmse = np.sqrt(-best_cv_score) if SCORING == 'neg_root_mean_squared_error' else best_cv_score
        logging.info(f"Лучшее значение RMSE на CV (валидация Optuna): {best_cv_rmse:.5f}")
        logging.info("Лучшие гиперпараметры:")
        for key, value in best_params.items(): logging.info(f"  {key}: {value}")

        # Обучаем финальную модель на ВСЕХ отобранных трейн данных
        final_pipeline = train_final_model(best_params, X_train_selected, y_full_train)

        # Сохраняем финальную модель
        try:
            joblib.dump(final_pipeline, OUTPUT_MODEL_PATH)
            logging.info(f"Финальный пайплайн сохранен в: {OUTPUT_MODEL_PATH}")
        except Exception as e: logging.error(f"Ошибка при сохранении финальной модели: {e}")

        # Оцениваем на локальном ОТОБРАННОМ тесте
        evaluate_model(final_pipeline, X_test_local, y_test_local)

        # --- 2.6 Создание Финального Сабмишена ---
        create_submission(
            pipeline=final_pipeline,
            test_data_path=TEST_DATA_PATH, # Путь к ПОЛНОМУ тестовому файлу с РАСШИРЕННЫМИ признаками
            selected_feature_names=selected_feature_names_final, # Список ОТОБРАННЫХ имен
            sample_submission_path=SAMPLE_SUBMISSION_PATH,
            output_path=OUTPUT_SUBMISSION_PATH,
            id_col=ID_COLUMN,
            target_col=TARGET_COLUMN
        )
    else:
        logging.warning("Optuna не нашла лучшего триала. Финальная модель не обучена, сабмишен не создан.")

    logging.info("--- СКРИПТ HPO LGBM (Extended + FS) ЗАВЕРШЕН ---")

### Остановка скрипта вручную.
- Результат продолжительное время не улучшался.
- Оставшееся время оценивалось предположительно в 70 часов

---

### СТЕКИНГ (SVM + LGBM + GNN)

In [ ]:
# -*- coding: utf-8 -*-
# --- ЕДИНЫЙ СКРИПТ ДЛЯ СТЕКИНГА (SVM + LGBM + GNN) v2 ---
import warnings
import os
import time
import joblib
import shutil

import pandas as pd
import numpy as np

# Визуализация (опционально)
# import matplotlib.pyplot as plt
# import seaborn as sns

# RDKit
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, AllChem, rdPartialCharges
RDLogger.DisableLog('rdApp.*')

# Scikit-learn
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import Ridge # Мета-модель
from sklearn.metrics import mean_squared_error, r2_score

# LightGBM
import lightgbm as lgb

# PyTorch & PyG (для GNN)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, GATv2Conv, global_add_pool, global_mean_pool, global_max_pool
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Optuna (для мета-модели)
import optuna

# Утилиты
# Определяем tqdm в зависимости от среды
try:
    from tqdm.notebook import tqdm as tqdm_notebook
    tqdm_func = tqdm_notebook
    print("Using tqdm.notebook")
except ImportError:
    from tqdm import tqdm as tqdm_base
    tqdm_func = tqdm_base
    print("Using standard tqdm")
from pathlib import Path
import logging
import sys # Для sys.exit()

# --- 0. Настройка и Конфигурация ---
# Игнорирование предупреждений
warnings.filterwarnings('ignore', category=UserWarning, message='.*X does not have valid feature names.*')
# warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", message="TypedStorage is deprecated")
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Основные пути к файлам (!!! ПРОВЕРЬ ИХ !!!)
SOURCE_TRAIN_DATA_PATH = 'train_data_clean_rdkit (2).csv' # Файл с SMILES, LogP (БЕЗ ID)
SOURCE_TEST_DATA_PATH = 'test_data.csv'                 # Файл с SMILES (и ID?, проверим) для теста
SAMPLE_SUBMISSION_PATH = 'final_sample_submission80.csv' # Пример сабмишена (с ID)

SVM_LGBM_TRAIN_FEATURES_PATH = 'final_train_data_04_14_201_new_alex.csv' # 201 признак + LogP для трейна
SVM_LGBM_TEST_FEATURES_PATH = 'final_test_data_04_14_201_new_alex.csv'  # 201 признак для теста

# Параметры
TARGET_COLUMN = 'LogP'
SMILES_COLUMN = 'SMILES'
ID_COLUMN = 'ID' # Будем генерировать эту колонку, если ее нет
SPLIT_RANDOM_STATE = 42
VALIDATION_SET_SIZE = 0.2 # Доля данных для валидации (OOF)

# Настройки GNN
GNN_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"GNN using device: {GNN_DEVICE}")
GNN_BATCH_SIZE = 64
GNN_EPOCHS = 115
GNN_FEATURE_DIMS = (16, 9, 219)

# Настройки Мета-модели (Ridge + Optuna)
META_MODEL_CV_FOLDS = 5
META_MODEL_OPTUNA_TRIALS = 50
META_MODEL_OPTUNA_TIMEOUT = 300 # Секунды

# Пути для сохранения промежуточных и финальных результатов
OUTPUT_DIR = Path('./stacking_output_v2') # Используем v2 для новой версии
OUTPUT_DIR.mkdir(exist_ok=True)


# --- 1. Лучшие Гиперпараметры Базовых Моделей ---
# (Оставляем без изменений)
BEST_SVM_PARAMS = { 'variance_threshold': 0.017612840487812365, 'kernel': 'rbf', 'C': 2.772897695046977, 'gamma_rbf': 'scale' }
BEST_LGBM_PARAMS = { 'scaler': 'None', 'use_variance_threshold': True, 'variance_threshold': 0.014620133713184701, 'learning_rate': 0.04287683192058186, 'n_estimators': 1954, 'num_leaves': 95, 'max_depth': 11, 'reg_alpha': 2.0709243779799007e-07, 'reg_lambda': 4.906962698572223e-06, 'colsample_bytree': 0.5094276601924259, 'subsample': 0.5613723967905625, 'min_child_samples': 5, 'objective': 'regression_l1', 'metric': 'rmse', 'random_state': SPLIT_RANDOM_STATE, 'n_jobs': -1 }
BEST_HYPERPARAMS_GNN = { 'gin_hidden': 128, 'n_layers': 5, 'desc_hidden': 1024, 'dropout': 0.45, 'activation': 'gelu', 'pooling': 'add', 'gnn_type': 'GINE', 'gat_heads': 4, 'train_eps': True, 'final_mlp_layers': 2, 'norm_type': 'LayerNorm', 'clip_grad_norm': 1.0, 'lr': 0.000518107585097511, 'weight_decay': 0.0004637649821784796, 'optimizer': 'AdamW', 'loss_delta': 1.3507041013870287, 'scheduler_T0': 12, 'scheduler_T_mult': 2 }


# --- 2. Классы и Функции GNN (С ИСПРАВЛЕНИЕМ SMARTS) ---

# Функция вычисления дескрипторов v2
def compute_descriptors_v2(smiles: str) -> tuple[Optional[np.ndarray], Optional[list]]:
    # (Без изменений)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return None, None
        all_desc_values = []; desc_names = []
        for desc_name, desc_func in Descriptors.descList:
            try: val = desc_func(mol); all_desc_values.append(float(val) if isinstance(val, (int, float)) and not (np.isnan(val) or np.isinf(val)) else np.nan)
            except Exception: all_desc_values.append(np.nan)
            desc_names.append(desc_name)
        try:
            rdPartialCharges.ComputeGasteigerCharges(mol); all_desc_values.append(Descriptors.MaxAbsPartialCharge(mol)); desc_names.append("MaxAbsPartialCharge"); all_desc_values.append(Descriptors.MinAbsPartialCharge(mol)); desc_names.append("MinAbsPartialCharge")
        except Exception: all_desc_values.extend([np.nan, np.nan]); desc_names.extend(["MaxAbsPartialCharge", "MinAbsPartialCharge"])
        descriptors = np.array(all_desc_values, dtype=np.float32)
        return descriptors, desc_names
    except Exception: return None, None

class LogPInMemoryDataset(InMemoryDataset):
    # SMARTS компиляция (Без изменений)
    _acid_query: Optional[Chem.Mol] = None
    _base_query: Optional[Chem.Mol] = None
    _smarts_compiled = False
    try:
        # ... (код компиляции SMARTS) ...
        _acid_smarts = '[CX3](=O)[OX1H0-,OX2H1]'
        _base_smarts = '[#7+0;!$(*~[#6]=[#7,#8,#16]);!$(*~[#16]=[#8]);!$(*~[#7]=[#8])],[#7+1X4]'
        _acid_query = Chem.MolFromSmarts(_acid_smarts)
        _base_query = Chem.MolFromSmarts(_base_smarts)
        if _acid_query is not None and _base_query is not None:
             _smarts_compiled = True
             # logging.info("GNN Dataset: Charge/Zwitterion SMARTS patterns compiled successfully.")
        else: _smarts_compiled = False; logging.error("GNN Dataset: Failed to compile SMARTS.")
    except Exception as e: logging.error(f"GNN Dataset: Unexpected error compiling SMARTS: {e}."); _smarts_compiled = False

    def __init__(self, root, smiles_list, logp_list, fit_scaler=False, scaler=None,
                 transform=None, pre_transform=None, pre_filter=None):

        self.root_path = Path(root)
        # <<< ИЗМЕНЕНИЕ: Устанавливаем атрибуты ДО вызова super() >>>
        self.smiles_list = smiles_list
        self.logp_list = logp_list
        self.fit_scaler = fit_scaler
        # Если scaler не передан, создаем новый. Он будет обучен в process(), если fit_scaler=True
        self.scaler = scaler if scaler is not None else StandardScaler()
        self._processed_feature_dims = None # Инициализируем здесь

        # Удаляем старые данные, если они есть, ДО вызова super(), чтобы process() точно запустился
        if self.root_path.exists() and not self.processed_paths_exist():
            logging.warning(f"GNN Dataset: Root exists but processed files not found. Removing {self.root_path}")
            shutil.rmtree(self.root_path)
        elif self.root_path.exists() and self.processed_paths_exist():
             logging.info(f"GNN Dataset: Processed files found in {self.root_path}. Skipping process().")


        # <<< ИЗМЕНЕНИЕ: super() вызывается ПОСЛЕ установки атрибутов >>>
        super().__init__(root, transform, pre_transform, pre_filter)

        # <<< ИЗМЕНЕНИЕ: Загружаем данные только если process() НЕ был вызван >>>
        # process() вызывается внутри super().__init__(), если файлы не найдены.
        # Если файлы были найдены, super() их не трогает, и нам нужно загрузить их здесь.
        if not hasattr(self, 'data') or self.data is None: # Проверяем, загрузил ли super() данные
            try:
                # Используем weights_only=False
                logging.info(f"GNN Dataset: Loading pre-processed data from {self.processed_paths[0]}...")
                self.data, self.slices = torch.load(self.processed_paths[0], map_location='cpu', weights_only=False)

                # Загружаем scaler, если он не был передан и файл существует
                scaler_path = self.root_path / 'scaler.pkl'
                if scaler is None and scaler_path.exists():
                    self.scaler = joblib.load(scaler_path)
                    # logging.info(f"GNN Dataset: Loaded scaler from {scaler_path}")
                elif scaler is None and not scaler_path.exists():
                     logging.error(f"GNN Dataset: Processed data loaded, but scaler file {scaler_path} not found and no scaler provided.")
                     # raise RuntimeError("Scaler missing for pre-processed data.") # Можно сделать строже

            except FileNotFoundError:
                 logging.error(f"GNN Dataset: Processed file {self.processed_paths[0]} should exist but not found.")
                 raise
            except Exception as e:
                 logging.error(f"GNN Dataset: Error loading existing processed data/scaler: {e}")
                 raise


    @property
    def raw_file_names(self) -> List[str]: return []
    @property
    def processed_file_names(self) -> List[str]: return ['data.pt']
    @property
    def processed_paths(self) -> List[str]: return [str(self.root_path / self.processed_file_names[0])]

    # <<< ИЗМЕНЕНИЕ: Добавляем проверку существования файлов >>>
    def processed_paths_exist(self) -> bool:
        return all(os.path.exists(p) for p in self.processed_paths)

    def download(self): pass

    # Методы _get_mol, _get_atom_features, _get_bond_features (Без изменений)
    # ...
    def _get_mol(self, smiles: str) -> Optional[Chem.Mol]:
        # ... (код без изменений) ...
        try: return Chem.MolFromSmiles(smiles)
        except Exception: return None

    def _get_atom_features(self, atom: Chem.Atom) -> List[float]:
        # ... (код без изменений) ...
        hybridization_map = { Chem.rdchem.HybridizationType.SP: [1,0,0,0,0], Chem.rdchem.HybridizationType.SP2: [0,1,0,0,0], Chem.rdchem.HybridizationType.SP3: [0,0,1,0,0], Chem.rdchem.HybridizationType.SP3D: [0,0,0,1,0], Chem.rdchem.HybridizationType.SP3D2: [0,0,0,0,1], Chem.rdchem.HybridizationType.UNSPECIFIED: [0,0,0,0,0], Chem.rdchem.HybridizationType.OTHER: [0,0,0,0,0] }
        hybridization = hybridization_map.get(atom.GetHybridization(), [0,0,0,0,0])
        chiral_tag = atom.GetChiralTag(); chiral_map = { Chem.rdchem.ChiralType.CHI_UNSPECIFIED: [1,0,0], Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW: [0,1,0], Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW: [0,0,1] }; chirality = chiral_map.get(chiral_tag, [1,0,0])
        features = [ float(atom.GetAtomicNum()), float(atom.GetDegree()), float(atom.GetTotalNumHs()), float(atom.GetImplicitValence()), float(atom.GetIsAromatic()), float(atom.GetFormalCharge()), float(atom.IsInRing()) ]
        try: gasteiger_charge = float(atom.GetProp('_GasteigerCharge')); gasteiger_charge = 0.0 if np.isnan(gasteiger_charge) or np.isinf(gasteiger_charge) else gasteiger_charge
        except Exception: gasteiger_charge = 0.0
        features.append(gasteiger_charge); features.extend(hybridization); features.extend(chirality)
        return features

    def _get_bond_features(self, bond: Chem.Bond) -> List[float]:
        # ... (код без изменений) ...
        bond_type = bond.GetBondTypeAsDouble(); is_conjugated = float(bond.GetIsConjugated()); is_in_ring = float(bond.IsInRing()); stereo = bond.GetStereo()
        stereo_map = { Chem.rdchem.BondStereo.STEREONONE: [1,0,0,0,0,0], Chem.rdchem.BondStereo.STEREOANY: [0,1,0,0,0,0], Chem.rdchem.BondStereo.STEREOZ: [0,0,1,0,0,0], Chem.rdchem.BondStereo.STEREOE: [0,0,0,1,0,0], Chem.rdchem.BondStereo.STEREOCIS: [0,0,0,0,1,0], Chem.rdchem.BondStereo.STEREOTRANS: [0,0,0,0,0,1] }
        bond_stereo = stereo_map.get(stereo, [1,0,0,0,0,0]); features = [bond_type, is_conjugated, is_in_ring]; features.extend(bond_stereo)
        return features

    # Метод process (Без изменений в логике вычислений, но теперь он будет вызван корректно)
    def process(self):
        # ... (весь код метода process остается таким же, как в предыдущей версии) ...
        logging.info(f"GNN Dataset: Processing data for {self.root}...")
        data_list = []; raw_descriptors_list = []; valid_indices = []; descriptor_names = None
        pbar_desc = tqdm_func(self.smiles_list, desc=f"GNN Descriptors ({self.root.name})", leave=False)
        for i, smi in enumerate(pbar_desc):
            desc, names = compute_descriptors_v2(smi)
            if desc is not None:
                raw_descriptors_list.append(desc); valid_indices.append(i)
                if descriptor_names is None and names is not None: descriptor_names = names
        if not raw_descriptors_list: raise ValueError("GNN Dataset: No valid descriptors computed.")
        computed_descriptors_np = np.array(raw_descriptors_list, dtype=np.float32)
        nan_mask = np.isnan(computed_descriptors_np); num_nan = np.sum(nan_mask)
        if num_nan > 0: logging.info(f"GNN Dataset: Imputing {num_nan} NaN values in descriptors with 0.0")
        computed_descriptors_np[nan_mask] = 0.0

        if self.fit_scaler:
            self.scaler.fit(computed_descriptors_np)
            scaler_path = self.root_path / 'scaler.pkl'
            joblib.dump(self.scaler, scaler_path)
            logging.info(f"GNN Dataset: Scaler fitted and saved to {scaler_path}")
        elif not hasattr(self.scaler, 'mean_'): raise ValueError("GNN Dataset: Scaler needs to be fitted or provided when fit_scaler=False.")

        computed_descriptors_scaled = self.scaler.transform(computed_descriptors_np)
        num_descriptor_features = computed_descriptors_scaled.shape[1]
        num_atom_f, num_edge_f = -1, -1
        pbar_graph = tqdm_func(valid_indices, desc=f"GNN Graphs ({self.root.name})", leave=False)
        skipped_graphs = 0
        for idx, orig_idx in enumerate(pbar_graph):
            smi = self.smiles_list[orig_idx]; mol = self._get_mol(smi)
            if not mol: skipped_graphs += 1; continue
            try:
                try: rdPartialCharges.ComputeGasteigerCharges(mol)
                except Exception: pass
                atom_features_list = [self._get_atom_features(atom) for atom in mol.GetAtoms()]
                if not atom_features_list: skipped_graphs += 1; continue
                expected_atom_len = GNN_FEATURE_DIMS[0]
                if any(len(f) != expected_atom_len for f in atom_features_list):
                    logging.warning(f"GNN: Atom feature length inconsistency for SMILES {smi}. Skipping.")
                    skipped_graphs += 1; continue
                atom_features = torch.tensor(atom_features_list, dtype=torch.float32)
                if num_atom_f == -1: num_atom_f = atom_features.shape[1]

                edge_indices, edge_attrs = [], []
                if mol.GetNumBonds() > 0:
                    expected_bond_len = GNN_FEATURE_DIMS[1]
                    for bond in mol.GetBonds():
                        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                        bond_features = self._get_bond_features(bond)
                        if len(bond_features) != expected_bond_len:
                             logging.warning(f"GNN: Bond feature length inconsistency for SMILES {smi}. Using zeros.")
                             bond_features = [0.0] * expected_bond_len
                        edge_indices.extend([(i, j), (j, i)])
                        edge_attrs.extend([bond_features, bond_features])
                    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
                    edge_attr = torch.tensor(edge_attrs, dtype=torch.float32)
                    if num_edge_f == -1: num_edge_f = edge_attr.shape[1]
                else:
                    edge_index = torch.empty((2, 0), dtype=torch.long)
                    current_num_edge_f = num_edge_f if num_edge_f != -1 else GNN_FEATURE_DIMS[1]
                    edge_attr = torch.empty((0, current_num_edge_f), dtype=torch.float32)

                desc_tensor = torch.tensor(computed_descriptors_scaled[idx], dtype=torch.float32)
                graph_data = Data(x=atom_features, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor([self.logp_list[orig_idx]], dtype=torch.float32), descriptors=desc_tensor.unsqueeze(0), smiles=smi)
                if self.pre_filter is not None and not self.pre_filter(graph_data): skipped_graphs += 1; continue
                if self.pre_transform is not None: graph_data = self.pre_transform(graph_data)
                data_list.append(graph_data)
            except Exception as e: logging.warning(f"GNN Dataset: Error processing graph for SMILES {smi}: {e}"); skipped_graphs += 1
        if num_edge_f == -1: num_edge_f = 0
        if not data_list: raise ValueError("GNN Dataset: No graphs were successfully created.")
        if skipped_graphs > 0: logging.warning(f"GNN Dataset: Skipped {skipped_graphs} graphs during processing.")

        # Важно: Устанавливаем атрибут _processed_feature_dims в конце process()
        if num_atom_f != GNN_FEATURE_DIMS[0] or num_edge_f != GNN_FEATURE_DIMS[1] or num_descriptor_features != GNN_FEATURE_DIMS[2]:
             logging.warning(f"GNN Dataset: Final feature dimension mismatch! Expected {GNN_FEATURE_DIMS}, Got ({num_atom_f}, {num_edge_f}, {num_descriptor_features}). Check config/data.")
        self._processed_feature_dims = (num_atom_f, num_edge_f, num_descriptor_features) # <<< УСТАНОВКА АТРИБУТА

        data, slices = self.collate(data_list)
        logging.info(f"GNN Dataset: Saving {len(data_list)} processed graphs to {self.processed_paths[0]}...")
        # <<< ИЗМЕНЕНИЕ: Используем weights_only=False при сохранении тоже (на всякий случай) >>>
        # Хотя по умолчанию для torch.save это False, но для явности укажем
        torch.save((data, slices), self.processed_paths[0], _use_new_zipfile_serialization=False) # _use_new_zipfile... для совместимости?
        logging.info(f"GNN Dataset: Processed feature dimensions: {self._processed_feature_dims}")


    # Метод get_scaler (Без изменений)
    def get_scaler(self):
        # ... (код без изменений) ...
        if not hasattr(self.scaler, 'mean_'):
            scaler_path = self.root_path / 'scaler.pkl'
            if scaler_path.exists(): self.scaler = joblib.load(scaler_path)
            else: raise RuntimeError("GNN Dataset: Scaler not fitted/found.")
        return self.scaler

    # Метод get_feature_dims (Модифицирован для большей надежности)
    def get_feature_dims(self):
        # Сначала пытаемся использовать сохраненный атрибут
        if hasattr(self, '_processed_feature_dims') and self._processed_feature_dims is not None:
             # Дополнительная проверка, что все значения корректны
             if all(isinstance(d, int) and d >= 0 for d in self._processed_feature_dims):
                  return self._processed_feature_dims

        # Если атрибут не установлен или некорректен, пытаемся определить по данным
        logging.warning("GNN Dataset: _processed_feature_dims not set or invalid. Attempting to determine from data[0].")
        if hasattr(self, 'data') and self.data is not None and len(self) > 0:
            try:
                d = self.get(0)
                num_atom_f = d.x.shape[1] if hasattr(d, 'x') and d.x is not None else -1
                num_edge_f = d.edge_attr.shape[1] if hasattr(d, 'edge_attr') and d.edge_attr is not None and d.edge_attr.nelement() > 0 else 0 # 0 если нет связей
                num_desc_f = d.descriptors.shape[1] if hasattr(d, 'descriptors') and d.descriptors is not None else -1

                if num_atom_f >= 0 and num_edge_f >= 0 and num_desc_f >= 0:
                    self._processed_feature_dims = (num_atom_f, num_edge_f, num_desc_f)
                    logging.info(f"GNN Dataset: Determined feature dims from data[0]: {self._processed_feature_dims}")
                    return self._processed_feature_dims
                else:
                     logging.error(f"GNN Dataset: Failed to determine valid dims from data[0]. Got ({num_atom_f}, {num_edge_f}, {num_desc_f})")
                     return None # Или вернуть GNN_FEATURE_DIMS как фоллбэк?

            except Exception as e:
                logging.error(f"GNN Dataset: Error determining dims from data[0]: {e}")
                return None
        else:
            logging.error("GNN Dataset: Cannot determine dims - data not loaded or empty.")
            return None

# Класс модели LogPPredictorV2
class LogPPredictorV2(nn.Module):
    # (Без изменений, как в предыдущем блоке кода)
    def __init__(self, num_atom_features, num_edge_features, num_descriptor_features, gnn_type='GINE', gat_heads=4, train_eps=True, final_mlp_layers=2, norm_type='LayerNorm', gin_hidden=256, n_layers=4, desc_hidden=512, dropout=0.4, activation_fn='relu', pooling_fn='add'):
        super().__init__()
        self.num_atom_features = num_atom_features; self.num_edge_features = num_edge_features; self.num_descriptor_features = num_descriptor_features; self.gnn_type = gnn_type
        try: self.activation = getattr(F, activation_fn)
        except AttributeError: raise ValueError(f"Activation '{activation_fn}' not found.")
        try: NormLayer = getattr(nn, norm_type)
        except AttributeError: raise ValueError(f"Normalization '{norm_type}' not found.")
        self.pooling_fn_name = pooling_fn
        desc_output_dim = desc_hidden // 2
        self.desc_mlp = nn.Sequential( nn.Linear(self.num_descriptor_features, desc_hidden), NormLayer(desc_hidden), nn.SiLU(), nn.Dropout(dropout), nn.Linear(desc_hidden, desc_output_dim), NormLayer(desc_output_dim), nn.SiLU(), nn.Dropout(dropout / 2) )
        self.convs = nn.ModuleList(); self.batch_norms = nn.ModuleList(); self.skip_lins = nn.ModuleList()
        gnn_out_dim = gin_hidden; in_channels = self.num_atom_features
        for i in range(n_layers):
            current_edge_dim_init = self.num_edge_features if self.num_edge_features > 0 else None
            if self.gnn_type == 'GINE':
                mlp = nn.Sequential( nn.Linear(in_channels, gin_hidden), nn.ReLU(), nn.Linear(gin_hidden, gin_hidden), nn.BatchNorm1d(gin_hidden) )
                self.convs.append(GINEConv(mlp, edge_dim=current_edge_dim_init, train_eps=train_eps)); gnn_out_dim = gin_hidden
            elif self.gnn_type == 'GATv2':
                self.convs.append(GATv2Conv( in_channels, gin_hidden // gat_heads, heads=gat_heads, dropout=dropout*0.5, edge_dim=current_edge_dim_init, concat=True )); gnn_out_dim = gin_hidden
            else: raise ValueError(f"Unsupported gnn_type: {self.gnn_type}")
            self.batch_norms.append(nn.BatchNorm1d(gnn_out_dim))
            if in_channels != gnn_out_dim: self.skip_lins.append(nn.Linear(in_channels, gnn_out_dim))
            else: self.skip_lins.append(nn.Identity())
            in_channels = gnn_out_dim
        self.attention_lin = nn.Linear(gnn_out_dim + desc_output_dim, 1); final_input_dim = gnn_out_dim + desc_output_dim; final_layers = []; current_dim = final_input_dim
        for i in range(final_mlp_layers):
            out_dim = current_dim // 2 if i < final_mlp_layers - 1 else 1; out_dim = max(1, out_dim)
            final_layers.append(nn.Linear(current_dim, out_dim))
            if i < final_mlp_layers - 1: final_layers.extend([NormLayer(out_dim), nn.SiLU(), nn.Dropout(dropout / (1.5 + i*0.5))])
            current_dim = out_dim
        self.final = nn.Sequential(*final_layers); self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if m is self: continue
            elif isinstance(m, nn.Linear):
                try: # Kaiming for ReLU-like activations
                     nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                except ValueError: # Fallback for others (e.g., SiLU, GELU might prefer Xavier)
                     nn.init.xavier_normal_(m.weight)
                if m.bias is not None: nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm1d, nn.LayerNorm)):
                 if m.weight is not None: nn.init.ones_(m.weight)
                 if m.bias is not None: nn.init.zeros_(m.bias)


    def forward(self, data):
        x, edge_index, edge_attr, descriptors, batch = data.x, data.edge_index, data.edge_attr, data.descriptors, data.batch
        # Handle cases where edge_attr might be None or empty during inference/processing issues
        current_edge_attr = edge_attr if self.num_edge_features > 0 and edge_attr is not None and edge_attr.numel() > 0 else None

        x_skip = x
        for i in range(len(self.convs)):
            x_identity = self.skip_lins[i](x_skip)
            # Pass edge_attr only if it's valid
            edge_attr_to_pass = current_edge_attr if self.convs[i].__class__.__name__ in ['GINEConv', 'GATv2Conv'] else None # Adapt based on conv needs edge_attr
            try:
                 x_conv = self.convs[i](x, edge_index, edge_attr=edge_attr_to_pass)
            except TypeError as e: # Catch if edge_attr is passed when not needed
                 if 'edge_attr' in str(e):
                      x_conv = self.convs[i](x, edge_index)
                 else: raise e
            x = x_conv + x_identity
            x = self.batch_norms[i](x)
            x = self.activation(x)
            x_skip = x

        if self.pooling_fn_name == 'add': x_graph = global_add_pool(x, batch)
        elif self.pooling_fn_name == 'mean': x_graph = global_mean_pool(x, batch)
        elif self.pooling_fn_name == 'max': x_graph = global_max_pool(x, batch)
        else: raise ValueError(f"Unsupported pooling: {self.pooling_fn_name}")

        num_graphs = data.num_graphs
        if descriptors.shape[0] != num_graphs:
            try: descriptors = descriptors.view(num_graphs, self.num_descriptor_features)
            except RuntimeError as e: raise ValueError(f"Desc batch mismatch: {e}") from e
        x_desc = self.desc_mlp(descriptors); combined_features = torch.cat([x_graph, x_desc], dim=1)
        attention_scores = torch.sigmoid(self.attention_lin(combined_features))
        gated_graph = attention_scores * x_graph; gated_desc = (1 - attention_scores) * x_desc
        combined_gated = torch.cat([gated_graph, gated_desc], dim=1); return self.final(combined_gated)


# --- 3. Функции Обучения и Предсказания Базовых Моделей ---

def train_predict_svm(X_train, y_train, X_predict, params, output_dir, model_suffix=''):
    """Обучает SVM и делает предсказания."""
    logging.info(f"Training SVM {model_suffix}...")
    start_time = time.time()
    try:
        preprocessor = Pipeline([
            ('variance_filter', VarianceThreshold(threshold=params['variance_threshold'])),
            ('scaler', StandardScaler())
        ])
        svr_model = SVR(kernel=params['kernel'], C=params['C'], gamma=params.get('gamma_rbf', 'scale'), cache_size=1000) # Добавим cache_size

        pipeline = Pipeline([
            ('preprocess', preprocessor),
            ('svm', svr_model)
        ])

        pipeline.fit(X_train, y_train)
        logging.info(f"SVM {model_suffix} training finished in {time.time() - start_time:.2f}s")

        # Сохраняем обученный пайплайн
        model_filename = f'svm_pipeline{model_suffix}.pkl'
        joblib.dump(pipeline, output_dir / model_filename)
        logging.info(f"SVM pipeline saved to {output_dir / model_filename}")

        logging.info(f"Predicting with SVM {model_suffix}...")
        predictions = pipeline.predict(X_predict)
        return predictions, pipeline
    except Exception as e:
        logging.error(f"Error during SVM {model_suffix} train/predict: {e}")
        raise # Передаем ошибку выше

def train_predict_lgbm(X_train, y_train, X_predict, params, output_dir, model_suffix=''):
    """Обучает LightGBM и делает предсказания."""
    logging.info(f"Training LightGBM {model_suffix}...")
    start_time = time.time()
    try:
        preprocessing_steps = []
        # scaler не используется согласно params
        if params.get('use_variance_threshold', False):
            preprocessing_steps.append(('variance_filter', VarianceThreshold(threshold=params['variance_threshold'])))

        # Убираем параметры препроцессинга из параметров LGBM
        lgbm_model_params = {k: v for k, v in params.items() if k not in ['scaler', 'use_variance_threshold', 'variance_threshold']}
        lgbm_model = lgb.LGBMRegressor(**lgbm_model_params)

        pipeline = Pipeline(preprocessing_steps + [('lgbm', lgbm_model)])
        # Передаем DataFrame в fit, чтобы LGBM видел имена фичей, если нужно
        pipeline.fit(X_train, y_train)
        logging.info(f"LightGBM {model_suffix} training finished in {time.time() - start_time:.2f}s")

        model_filename = f'lgbm_pipeline{model_suffix}.pkl'
        joblib.dump(pipeline, output_dir / model_filename)
        logging.info(f"LGBM pipeline saved to {output_dir / model_filename}")

        logging.info(f"Predicting with LightGBM {model_suffix}...")
        predictions = pipeline.predict(X_predict)
        return predictions, pipeline
    except Exception as e:
        logging.error(f"Error during LGBM {model_suffix} train/predict: {e}")
        raise

def train_predict_gnn(train_smiles, train_logp, predict_smiles, params, feature_dims, device, output_dir, mode='oof'):
    """Обучает GNN и делает предсказания. mode='oof' или 'full'."""
    logging.info(f"--- Starting GNN Task (mode: {mode}) ---")
    start_time_task = time.time()

    train_root = output_dir / f'gnn_data_{mode}_train'
    predict_root = output_dir / f'gnn_data_{mode}_predict'
    model_save_path = output_dir / f'gnn_model_{mode}.pth'
    scaler_save_path = output_dir / f'gnn_scaler_{mode}.pkl'

    actual_dims = None # Инициализируем переменную для размерностей

    try:
        # 1. Обработка трейн данных и обучение scaler'а
        logging.info(f"GNN ({mode}): Initializing train dataset ({len(train_smiles)} smiles)...")
        # Удаляем старую папку перед созданием, чтобы гарантировать запуск process()
        if train_root.exists():
             logging.warning(f"GNN ({mode}): Removing existing train data directory: {train_root}")
             shutil.rmtree(train_root)

        train_dataset = LogPInMemoryDataset(root=train_root, smiles_list=train_smiles, logp_list=train_logp, fit_scaler=True)
        scaler = train_dataset.get_scaler() # Получаем scaler
        logging.info(f"GNN ({mode}): Train dataset initialized. Root: {train_root}")

        # <<< ИЗМЕНЕНИЕ: Получаем размерности ПОСЛЕ инициализации >>>
        logging.info(f"GNN ({mode}): Determining feature dimensions...")
        actual_dims = train_dataset.get_feature_dims()
        if actual_dims is None or any(d < 0 for d in actual_dims): # Проверяем, что размеры корректны
             raise ValueError(f"Failed to determine valid feature dimensions from train_dataset. Got: {actual_dims}")
        logging.info(f"GNN ({mode}): Actual feature dimensions: {actual_dims}")
        # --- Конец изменения ---

        train_loader = DataLoader(train_dataset, batch_size=GNN_BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)

        # 2. Создание и обучение модели GNN
        logging.info(f"GNN ({mode}): Creating and training model...")
        num_atom_f, num_edge_f, num_desc_f = actual_dims # Используем полученные размеры

        model = LogPPredictorV2( # Передаем все параметры явно
            num_atom_features=num_atom_f, num_edge_features=num_edge_f, num_descriptor_features=num_desc_f,
            gnn_type=params.get('gnn_type', 'GINE'), gin_hidden=params['gin_hidden'], n_layers=params['n_layers'],
            gat_heads=params.get('gat_heads', 4), train_eps=params.get('train_eps', True),
            desc_hidden=params['desc_hidden'], final_mlp_layers=params.get('final_mlp_layers', 2),
            norm_type=params.get('norm_type', 'LayerNorm'), dropout=params['dropout'],
            activation_fn=params.get('activation', 'relu'), pooling_fn=params.get('pooling', 'add')
        ).to(device)

        optimizer_class = getattr(torch.optim, params['optimizer'])
        optimizer = optimizer_class(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=params['scheduler_T0'], T_mult=params.get('scheduler_T_mult', 2), eta_min=1e-6)
        criterion = nn.HuberLoss(delta=params['loss_delta'])

        start_time_train = time.time()
        model.train()
        pbar_epoch = tqdm_func(range(GNN_EPOCHS), desc=f"GNN Train ({mode})", leave=False)
        for epoch in pbar_epoch:
            # ... (внутренний цикл обучения без изменений) ...
            epoch_loss = 0; batch_count = 0
            for batch in train_loader:
                batch = batch.to(device); optimizer.zero_grad()
                try:
                    out = model(batch); loss = criterion(out, batch.y.view_as(out)); loss.backward()
                    clip_norm = params.get('clip_grad_norm')
                    if clip_norm is not None: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_norm)
                    optimizer.step(); epoch_loss += loss.item(); batch_count += 1
                except Exception as e: logging.warning(f"GNN Train Batch Error (Epoch {epoch+1}): {e}"); continue
            scheduler.step()
            if batch_count > 0:
                 avg_loss = epoch_loss / batch_count
                 pbar_epoch.set_postfix({'loss': f'{avg_loss:.4f}'})
            else:
                 logging.warning(f"GNN Epoch {epoch+1}: No batches processed.")


        logging.info(f"GNN ({mode}) training finished in {time.time() - start_time_train:.2f}s")
        torch.save(model.state_dict(), model_save_path)
        joblib.dump(scaler, scaler_save_path)
        logging.info(f"GNN model saved to {model_save_path}, scaler to {scaler_save_path}")

        # 3. Обработка данных для предсказания
        logging.info(f"GNN ({mode}): Processing prediction data ({len(predict_smiles)} smiles)...")
         # Удаляем старую папку перед созданием
        if predict_root.exists():
             logging.warning(f"GNN ({mode}): Removing existing prediction data directory: {predict_root}")
             shutil.rmtree(predict_root)
        dummy_logp = [0.0] * len(predict_smiles)
        predict_dataset = LogPInMemoryDataset(root=predict_root, smiles_list=predict_smiles, logp_list=dummy_logp, fit_scaler=False, scaler=scaler)
        predict_loader = DataLoader(predict_dataset, batch_size=GNN_BATCH_SIZE * 2, shuffle=False, num_workers=0)
        logging.info(f"GNN ({mode}): Prediction data processed.")

        # 4. Получение предсказаний
        # ... (код предсказания без изменений) ...
        logging.info(f"GNN ({mode}): Predicting...")
        model.eval()
        predictions_list = []
        pbar_pred = tqdm_func(predict_loader, desc=f"GNN Predict ({mode})", leave=False)
        with torch.no_grad():
            for batch in pbar_pred:
                batch = batch.to(device)
                try: pred = model(batch); predictions_list.extend(pred.cpu().numpy().flatten())
                except Exception as e:
                    logging.warning(f"GNN Predict Batch Error: {e}")
                    num_in_batch = batch.y.size(0); predictions_list.extend([np.nan] * num_in_batch); continue
        predictions = np.array(predictions_list)
        if np.isnan(predictions).any():
            num_nan = np.isnan(predictions).sum()
            logging.warning(f"GNN ({mode}): Found {num_nan} NaN predictions. Imputing with mean.")
            mean_pred = np.nanmean(predictions); fill_value = mean_pred if not np.isnan(mean_pred) else 0.0
            predictions[np.isnan(predictions)] = fill_value


        logging.info(f"--- GNN Task (mode: {mode}) finished in {time.time() - start_time_task:.2f}s ---")
        return predictions, model, scaler
    except Exception as e:
         logging.error(f"Error during GNN task (mode: {mode}): {e}")
         import traceback
         traceback.print_exc()
         raise

# --- 4. Основной Пайплайн Стекинга ---
def run_stacking():
    logging.info("--- STARTING STACKING PIPELINE v2 ---")

    # --- 4.1 Загрузка исходных данных (С ГЕНЕРАЦИЕЙ ID) ---
    logging.info("Loading source data...")
    try:
        df_train_source = pd.read_csv(SOURCE_TRAIN_DATA_PATH)
        df_test_source = pd.read_csv(SOURCE_TEST_DATA_PATH)
        df_sample_sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)

        # Проверка и генерация ID для трейна
        if ID_COLUMN not in df_train_source.columns:
            logging.warning(f"Column '{ID_COLUMN}' not found in train source. Generating ID from index.")
            df_train_source[ID_COLUMN] = df_train_source.index
        # Проверка остальных колонок трейна
        if SMILES_COLUMN not in df_train_source.columns or TARGET_COLUMN not in df_train_source.columns:
            raise ValueError(f"Missing '{SMILES_COLUMN}' or '{TARGET_COLUMN}' in train source.")

        # Проверка и генерация ID для теста
        if ID_COLUMN not in df_test_source.columns:
            logging.warning(f"Column '{ID_COLUMN}' not found in test source. Generating ID from index.")
            df_test_source[ID_COLUMN] = df_test_source.index
        # Проверка SMILES в тесте
        if SMILES_COLUMN not in df_test_source.columns:
             raise ValueError(f"Missing '{SMILES_COLUMN}' in test source.")
        # Проверка ID в sample submission
        if ID_COLUMN not in df_sample_sub.columns:
             raise ValueError(f"Missing '{ID_COLUMN}' in sample submission.")

        logging.info(f"Train source: {df_train_source.shape}, Test source: {df_test_source.shape}")
    except Exception as e: logging.error(f"Failed to load or process source data: {e}"); return

    # --- 4.2 Загрузка данных с признаками для SVM/LGBM ---
    logging.info("Loading feature data for SVM/LGBM...")
    try:
        df_train_features = pd.read_csv(SVM_LGBM_TRAIN_FEATURES_PATH)
        df_test_features = pd.read_csv(SVM_LGBM_TEST_FEATURES_PATH)

        # Определяем имена признаков (все колонки минус таргет)
        if TARGET_COLUMN not in df_train_features.columns:
             raise ValueError(f"Target column '{TARGET_COLUMN}' not found in train features file.")
        svm_lgbm_feature_cols = [col for col in df_train_features.columns if col != TARGET_COLUMN]
        if len(svm_lgbm_feature_cols) != 201:
             raise ValueError(f"Expected 201 features for SVM/LGBM, found {len(svm_lgbm_feature_cols)}")

        # Проверяем, есть ли колонка ID в тестовых признаках (иногда бывает)
        if ID_COLUMN in df_test_features.columns and len(df_test_features.columns) == 202:
             logging.info(f"Removing '{ID_COLUMN}' column from test features.")
             X_svm_lgbm_test_raw = df_test_features.drop(columns=[ID_COLUMN])
             # Убедимся, что порядок колонок совпадает с трейном
             X_svm_lgbm_test_raw = X_svm_lgbm_test_raw[svm_lgbm_feature_cols]
        elif len(df_test_features.columns) == 201:
             # Предполагаем, что это только признаки и порядок верный
             X_svm_lgbm_test_raw = df_test_features[svm_lgbm_feature_cols]
        else:
             raise ValueError(f"Unexpected number of columns in test features file: {len(df_test_features.columns)}")


        # --- СОПОСТАВЛЕНИЕ ПРИЗНАКОВ С ID/SMILES (Упрощенный вариант, если порядок совпадает) ---
        if len(df_train_source) != len(df_train_features) or len(df_test_source) != len(df_test_features):
             logging.error("Row count mismatch between source files and feature files. Cannot proceed with simple alignment.")
             # Здесь нужна была бы более сложная логика merge по SMILES или др. ключам, если порядок не совпадает.
             # Для простоты пока предполагаем, что порядок совпадает.
             return

        logging.info("Aligning features based on assumed matching row order...")
        # Добавляем ID из source к feature dataframes
        df_train_features_with_id = df_train_features.copy()
        df_train_features_with_id[ID_COLUMN] = df_train_source[ID_COLUMN].values

        df_test_features_with_id = X_svm_lgbm_test_raw.copy() # X_svm_lgbm_test_raw уже содержит только фичи
        df_test_features_with_id[ID_COLUMN] = df_test_source[ID_COLUMN].values

        logging.info(f"SVM/LGBM Train Features + ID: {df_train_features_with_id.shape}")
        logging.info(f"SVM/LGBM Test Features + ID: {df_test_features_with_id.shape}")

    except Exception as e: logging.error(f"Failed to load or process SVM/LGBM feature data: {e}"); return

    # --- 4.3 Единое разделение данных ---
    logging.info("Performing unified train/validation split...")
    # Разделяем ИСХОДНЫЙ DataFrame С ID
    train_ids, val_ids = train_test_split(
        df_train_source[ID_COLUMN].values, # Разделяем по ID
        test_size=VALIDATION_SET_SIZE,
        random_state=SPLIT_RANDOM_STATE,
    )

    # Создаем DataFrame'ы для трейна и валидации на основе ID
    train_base_df = df_train_source[df_train_source[ID_COLUMN].isin(train_ids)].copy()
    val_df = df_train_source[df_train_source[ID_COLUMN].isin(val_ids)].copy()
    y_val_true = val_df[TARGET_COLUMN].values
    np.save(OUTPUT_DIR / 'y_val_true.npy', y_val_true) # Сохраняем сразу

    # Отбираем признаки SVM/LGBM для OOF по ID
    X_svm_lgbm_train_base = df_train_features_with_id[df_train_features_with_id[ID_COLUMN].isin(train_ids)][svm_lgbm_feature_cols]
    y_svm_lgbm_train_base = df_train_features_with_id[df_train_features_with_id[ID_COLUMN].isin(train_ids)][TARGET_COLUMN]
    X_svm_lgbm_val = df_train_features_with_id[df_train_features_with_id[ID_COLUMN].isin(val_ids)][svm_lgbm_feature_cols]

    logging.info(f"Split complete: Train base size: {len(train_base_df)}, Validation size: {len(val_df)}")
    logging.info(f"SVM/LGBM features shapes: Train base {X_svm_lgbm_train_base.shape}, Val {X_svm_lgbm_val.shape}")
    if len(X_svm_lgbm_val) != len(val_df):
         logging.error("Mismatch between validation set size and selected SVM/LGBM features for validation!")
         return

    # --- 4.4 Генерация OOF Предсказаний ---
    logging.info("--- Generating OOF Predictions (Level 0) ---")
    oof_preds = {}
    # Словарь для хранения полных моделей/пайплайнов, обученных на ВСЕХ трейн данных
    full_trained_models = {}

    # SVM OOF
    try:
        svm_oof_preds, _ = train_predict_svm( # Пайплайн для OOF не сохраняем
            X_svm_lgbm_train_base.values, # Передаем NumPy
            y_svm_lgbm_train_base.values,
            X_svm_lgbm_val.values,
            BEST_SVM_PARAMS,
            OUTPUT_DIR,
            model_suffix='_oof'
        )
        oof_preds['svm'] = svm_oof_preds
        np.save(OUTPUT_DIR / 'svm_oof.npy', svm_oof_preds)
        logging.info(f"SVM OOF generation complete. Shape: {svm_oof_preds.shape}")
    except Exception as e: logging.error(f"SVM OOF generation failed: {e}"); return

    # LGBM OOF
    try:
        lgbm_oof_preds, _ = train_predict_lgbm(
            X_svm_lgbm_train_base, # Передаем DataFrame
            y_svm_lgbm_train_base,
            X_svm_lgbm_val,
            BEST_LGBM_PARAMS,
            OUTPUT_DIR,
            model_suffix='_oof'
        )
        oof_preds['lgbm'] = lgbm_oof_preds
        np.save(OUTPUT_DIR / 'lgbm_oof.npy', lgbm_oof_preds)
        logging.info(f"LGBM OOF generation complete. Shape: {lgbm_oof_preds.shape}")
    except Exception as e: logging.error(f"LGBM OOF generation failed: {e}"); return

    # GNN OOF
    try:
        gnn_oof_preds, _, _ = train_predict_gnn( # Модель и scaler для OOF не сохраняем в основной словарь
            train_base_df[SMILES_COLUMN].tolist(),
            train_base_df[TARGET_COLUMN].tolist(),
            val_df[SMILES_COLUMN].tolist(),
            BEST_HYPERPARAMS_GNN,
            GNN_FEATURE_DIMS,
            GNN_DEVICE,
            OUTPUT_DIR,
            mode='oof'
        )
        oof_preds['gnn'] = gnn_oof_preds
        np.save(OUTPUT_DIR / 'gnn_oof.npy', gnn_oof_preds)
        logging.info(f"GNN OOF generation complete. Shape: {gnn_oof_preds.shape}")
    except Exception as e: logging.error(f"GNN OOF generation failed: {e}"); return

    # Проверка размерностей OOF
    oof_lengths = {name: len(pred) for name, pred in oof_preds.items()}
    if not (oof_lengths['svm'] == oof_lengths['lgbm'] == oof_lengths['gnn'] == len(y_val_true)):
        logging.error(f"OOF predictions size mismatch! Lengths: {oof_lengths}, Expected: {len(y_val_true)}")
        return

    # --- 4.5 Обучение Мета-Модели (Ridge + Optuna) ---
    logging.info("--- Training Meta-Model (Level 1) ---")
    meta_model = None
    try:
        X_meta_train = pd.DataFrame({
            'svm_pred': oof_preds['svm'], 'lgbm_pred': oof_preds['lgbm'], 'gnn_pred': oof_preds['gnn'],
            'diff_svm_gnn': np.abs(oof_preds['svm'] - oof_preds['gnn']),
            'diff_lgbm_gnn': np.abs(oof_preds['lgbm'] - oof_preds['gnn']),
            'diff_svm_lgbm': np.abs(oof_preds['svm'] - oof_preds['lgbm']),
            'svm_pred_sq': oof_preds['svm']**2, 'lgbm_pred_sq': oof_preds['lgbm']**2, 'gnn_pred_sq': oof_preds['gnn']**2,
        })
        y_meta_train = y_val_true
        logging.info(f"Meta-features (train) created. Shape: {X_meta_train.shape}")

        def objective_ridge(trial):
            alpha = trial.suggest_float('alpha', 1e-4, 1e4, log=True)
            model = Ridge(alpha=alpha, random_state=SPLIT_RANDOM_STATE)
            kfold = KFold(n_splits=META_MODEL_CV_FOLDS, shuffle=True, random_state=trial.number)
            scores = cross_val_score(model, X_meta_train, y_meta_train, cv=kfold, scoring='neg_root_mean_squared_error', n_jobs=-1)
            return np.mean(scores)

        study_ridge = optuna.create_study(direction='maximize')
        logging.info(f"Optimizing Ridge alpha ({META_MODEL_OPTUNA_TRIALS} trials)...")
        study_ridge.optimize(objective_ridge, n_trials=META_MODEL_OPTUNA_TRIALS, timeout=META_MODEL_OPTUNA_TIMEOUT)

        if study_ridge.best_trial is None: raise RuntimeError("Optuna failed to find parameters for Ridge.")
        best_alpha = study_ridge.best_params['alpha']
        best_cv_rmse = -study_ridge.best_value
        logging.info(f"Ridge optimization complete. Best CV RMSE: {best_cv_rmse:.5f}, Best alpha: {best_alpha:.6f}")

        meta_model = Ridge(alpha=best_alpha, random_state=SPLIT_RANDOM_STATE)
        meta_model.fit(X_meta_train, y_meta_train)
        logging.info("Final Meta-Model (Ridge) trained.")
        joblib.dump(meta_model, OUTPUT_DIR / 'meta_model_ridge.pkl')
    except Exception as e: logging.error(f"Meta-Model training failed: {e}"); return

    # --- 4.6 Генерация Тестовых Предсказаний Базовых Моделей ---
    logging.info("--- Generating Test Predictions (Level 0) ---")
    test_preds = {}

    # Данные для обучения полных моделей
    X_train_full_svm_lgbm = df_train_features_with_id[svm_lgbm_feature_cols]
    y_train_full_svm_lgbm = df_train_features_with_id[TARGET_COLUMN]
    train_smiles_full = df_train_source[SMILES_COLUMN].tolist()
    train_logp_full = df_train_source[TARGET_COLUMN].tolist()
    # Тестовые данные
    X_test_svm_lgbm = df_test_features_with_id[svm_lgbm_feature_cols]
    test_smiles = df_test_source[SMILES_COLUMN].tolist()

    # SVM Test Preds (Обучение на полном трейне)
    try:
        logging.info("Training FULL SVM and predicting on test...")
        svm_test_preds, full_svm_pipeline = train_predict_svm(
            X_train_full_svm_lgbm.values, y_train_full_svm_lgbm.values, X_test_svm_lgbm.values,
            BEST_SVM_PARAMS, OUTPUT_DIR, model_suffix='_full'
        )
        test_preds['svm'] = svm_test_preds
        np.save(OUTPUT_DIR / 'svm_test_preds.npy', svm_test_preds)
        logging.info(f"SVM Test predictions generated. Shape: {svm_test_preds.shape}")
    except Exception as e: logging.error(f"SVM Full training/Test prediction failed: {e}"); return

    # LGBM Test Preds (Обучение на полном трейне)
    try:
        logging.info("Training FULL LGBM and predicting on test...")
        lgbm_test_preds, full_lgbm_pipeline = train_predict_lgbm(
            X_train_full_svm_lgbm, y_train_full_svm_lgbm, X_test_svm_lgbm,
            BEST_LGBM_PARAMS, OUTPUT_DIR, model_suffix='_full'
        )
        test_preds['lgbm'] = lgbm_test_preds
        np.save(OUTPUT_DIR / 'lgbm_test_preds.npy', lgbm_test_preds)
        logging.info(f"LGBM Test predictions generated. Shape: {lgbm_test_preds.shape}")
    except Exception as e: logging.error(f"LGBM Full training/Test prediction failed: {e}"); return

    # GNN Test Preds (Обучение на полном трейне)
    try:
        logging.info("Training FULL GNN and predicting on test...")
        gnn_test_preds, full_gnn_model, full_gnn_scaler = train_predict_gnn(
            train_smiles_full, train_logp_full, test_smiles,
            BEST_HYPERPARAMS_GNN, GNN_FEATURE_DIMS, GNN_DEVICE,
            OUTPUT_DIR, mode='full' # Используем другой режим/папку
        )
        test_preds['gnn'] = gnn_test_preds
        np.save(OUTPUT_DIR / 'gnn_test_preds.npy', gnn_test_preds)
        logging.info(f"GNN Test predictions generated. Shape: {gnn_test_preds.shape}")
        # Сохраняем полную модель GNN и scaler
        # torch.save(full_gnn_model.state_dict(), OUTPUT_DIR / 'gnn_model_full.pth') # Уже сохранено в функции
        # joblib.dump(full_gnn_scaler, OUTPUT_DIR / 'gnn_scaler_full.pkl') # Уже сохранено в функции
    except Exception as e: logging.error(f"GNN Full training/Test prediction failed: {e}"); return

    # Проверка размеров тестовых предсказаний
    if not (len(test_preds['svm']) == len(test_preds['lgbm']) == len(test_preds['gnn']) == len(df_test_source)):
        logging.error(f"Test predictions size mismatch! SVM:{len(test_preds['svm'])}, LGBM:{len(test_preds['lgbm'])}, GNN:{len(test_preds['gnn'])}, Expected:{len(df_test_source)}")
        return

    # --- 4.7 Финальное Предсказание Мета-Моделью ---
    logging.info("--- Generating Final Predictions (Level 1) ---")
    try:
        X_meta_test = pd.DataFrame({
            'svm_pred': test_preds['svm'], 'lgbm_pred': test_preds['lgbm'], 'gnn_pred': test_preds['gnn'],
            'diff_svm_gnn': np.abs(test_preds['svm'] - test_preds['gnn']),
            'diff_lgbm_gnn': np.abs(test_preds['lgbm'] - test_preds['gnn']),
            'diff_svm_lgbm': np.abs(test_preds['svm'] - test_preds['lgbm']),
            'svm_pred_sq': test_preds['svm']**2, 'lgbm_pred_sq': test_preds['lgbm']**2, 'gnn_pred_sq': test_preds['gnn']**2,
        })
        logging.info(f"Meta-features (test) created. Shape: {X_meta_test.shape}")

        # Загрузка мета-модели (если запускается отдельно)
        if meta_model is None:
            meta_model = joblib.load(OUTPUT_DIR / 'meta_model_ridge.pkl')
            logging.info("Reloaded meta-model.")

        final_predictions = meta_model.predict(X_meta_test)
        logging.info(f"Final predictions generated. Shape: {final_predictions.shape}")
    except Exception as e: logging.error(f"Final prediction failed: {e}"); return

    # --- 4.8 Формирование Сабмишена ---
    logging.info("--- Creating Submission File ---")
    try:
        # Используем ID из df_test_source, где мы его сгенерировали, если было нужно
        final_sub_ids = df_test_source[ID_COLUMN].values
        if len(final_sub_ids) != len(final_predictions):
             raise ValueError(f"Length mismatch between final predictions ({len(final_predictions)}) and test IDs ({len(final_sub_ids)})")

        submission_df = pd.DataFrame({ ID_COLUMN: final_sub_ids, TARGET_COLUMN: final_predictions })

        # --- Важно: Сопоставление с ID из Sample Submission ---
        # Загружаем sample submission ЕЩЕ РАЗ, чтобы гарантировать правильные ID и порядок
        df_sample_sub_final = pd.read_csv(SAMPLE_SUBMISSION_PATH)[[ID_COLUMN]] # Берем только ID
        # Объединяем наши предсказания с sample submission ID
        final_submission_df = df_sample_sub_final.merge(submission_df, on=ID_COLUMN, how='left')

        # Проверка на пропуски после merge (если ID не совпали)
        if final_submission_df[TARGET_COLUMN].isnull().any():
             logging.warning("NaN values found in final submission after merging with sample! Check ID matching.")
             # Можно попробовать заполнить пропуски средним или медианой предсказаний, но это плохой знак
             # final_submission_df[TARGET_COLUMN].fillna(np.median(final_predictions), inplace=True)

        # Проверка итогового размера
        if len(final_submission_df) != len(df_sample_sub):
            logging.warning(f"Final submission size ({len(final_submission_df)}) differs from sample submission size ({len(df_sample_sub)}).")

        submission_path = OUTPUT_DIR / 'submission_stacking_svm_lgbm_gnn_v2.csv'
        final_submission_df[[ID_COLUMN, TARGET_COLUMN]].to_csv(submission_path, index=False) # Сохраняем только 2 колонки
        logging.info(f"Submission file saved to: {submission_path}")
        logging.info("Submission head:\n" + final_submission_df.head().to_string())

    except Exception as e: logging.error(f"Failed to create submission file: {e}"); return

    logging.info("--- STACKING PIPELINE FINISHED SUCCESSFULLY ---")

# --- Запуск Пайплайна ---
if __name__ == "__main__":
    if 'ipykernel' not in sys.modules:
        tqdm_func = tqdm_base
        print("Running as script, using standard tqdm.")
    else:
         # Убедимся, что tqdm_func определен и в ноутбуке
         try: from tqdm.notebook import tqdm as tqdm_notebook; tqdm_func = tqdm_notebook
         except ImportError: from tqdm import tqdm as tqdm_base; tqdm_func = tqdm_base

    run_stacking()